In [1]:
import os

import gurobipy as gp
from gurobipy import GRB
import numpy as np

# Defining functions and formulations

In [2]:
# set index r for 2nd and 3rd formulations
# r = np.random.randint(n)
r2 = 5 
r3 = 5

In [3]:
def read_data(dir='data'):
    data = dict()
    files = os.listdir(dir)  # Lists all files and folders
    # Filter only files (exclude directories)
    files = [f for f in files if os.path.isfile(os.path.join(dir, f))]
    for f in files:
        name = f.split('.')
        if name[1] == 'txt':
            name = name[0]
            # reading the file
            with open(os.path.join(dir, f), "r") as ff:
                lines = ff.readlines()
            
            # Extract number of nodes
            num_nodes = int(lines[0].strip())
            # Load distance matrix
            distance_matrix = np.loadtxt(lines[1:], dtype=int)
            data[name] = distance_matrix
    return data

In [4]:
def print_results(model:gp.Model):
    if model.status == GRB.OPTIMAL:
        logFile = model.getParamInfo('logFile')[2]
        with open(logFile, 'a') as log:
            print("Optimal Solution:")
            log.write("\n \t Optimal Solution:\n")
            for v in model.getVars():
                if v.X != 0:  # 1 for x and f values and integer numbers for u values
                    print(f'{v.VarName}: {v.X}')
                    log.write(f'\t{v.VarName}: {v.X} \n')
            print(f'objective value: {model.Objval}')
            log.write(f'\tobjective value: {model.Objval}')
    else:
        print("No optimal solution found.")


In [5]:
def get_f0(data:np.array):
    n = data.shape[0]
    m = gp.Model('f0')
    
    # add variables
    x = m.addVars(n, n, vtype=GRB.BINARY, name='x')  # 1e
    
    m.addConstrs((gp.quicksum(x[i,j] for j in range(n))==1 for i in range(n)),
                 name='1b') 
    m.addConstrs((gp.quicksum(x[i,j] for i in range(n))==1 for j in range(n)),
                 name='1c')
    m.addConstrs((x[i, i] == 0 for i in range(n)),
                 name='1d')

    m.setObjective(gp.quicksum(data[i,j]*x[i,j] 
                               for i in range(n) 
                               for j in range(n)),
                   GRB.MINIMIZE)
    return m

In [6]:
def get_f2(data:np.array):
    n = data.shape[0]
    # similar to f0
    m = gp.Model('f2')
    x = m.addVars(n, n, vtype=GRB.BINARY, name='x')  # 1e
    m.addConstrs((gp.quicksum(x[i,j] for j in range(n))==1 for i in range(n)),
                 name='1b') 
    m.addConstrs((gp.quicksum(x[i,j] for i in range(n))==1 for j in range(n)),
                 name='1c')
    m.addConstrs((x[i, i] == 0 for i in range(n)),
                 name='1d')
    # new constrains for f2
    u = m.addVars(n, vtype=GRB.INTEGER, lb=1, name='u')
    r = r2
    
    m.addConstrs((u[i]-u[j]+1<=n*(1-x[i,j]) for i in range(n) for j in range(n) if i != r and j != r), name='3a')
    m.addConstr(u[r] == 1, name='3b')
    m.addConstrs((u[i]<=n for i in range(n) if i != r), name='3c1')
    m.addConstrs((u[i]>=2 for i in range(n) if i != r), name='3c2')  # added lower bound when defining 'u', do I need this now?
    
    m.setObjective(gp.quicksum(data[i,j]*x[i,j] 
                               for i in range(n) 
                               for j in range(n)),
                   GRB.MINIMIZE)
    return m


def run_f2(logfileName, params, datasets):
    for name,data in datasets.items():
        with gp.Env() as env:       
            with gp.Model(env=env) as model2:
                try:
                    m2 = get_f2(data)
                    m2.setParam("LogFile", f'logs/{logfileName}_{name}.log')
                    for key, value in params.items():
                        m2.setParam(key, value)
                    m2.optimize()
                    print_results(m2)
                except gp.GurobiError as e:
                    print('Error code ' + str(e.errno) + ': ' + str(e))
                except AttributeError:
                    print('Encountered an attribute error')

In [7]:
def get_f3(data:np.array):
    n = data.shape[0]
    # similar to f0
    m = gp.Model('f3')
    x = m.addVars(n, n, vtype=GRB.BINARY, name='x')  # 1e
    m.addConstrs((gp.quicksum(x[i,j] for j in range(n))==1 for i in range(n)),
                 name='1b') 
    m.addConstrs((gp.quicksum(x[i,j] for i in range(n))==1 for j in range(n)),
                 name='1c')
    m.addConstrs((x[i, i] == 0 for i in range(n)),
                 name='1d')
    # f3 constraints
    f = m.addVars(n, n, n, vtype=GRB.CONTINUOUS, lb=0, ub=1, name='f')
    r = r3
    
    m.addConstrs((gp.quicksum(f[v,r,j] for j in range(n) if j != r) - 
                  gp.quicksum(f[v,j,r] for j in range(n) if j != r) == 1  
                  for v in range(n) 
                  if v != r),
                 name='4a')
    
    m.addConstrs((gp.quicksum(f[v,i,j] for j in range(n) if j != i) - 
                  gp.quicksum(f[v,j,i] for j in range(n) if j != i) == 0  
                  for v in range(n) for i in range(n) 
                  if v != r and i != r and i != v), 
                 name='4b')
    
    m.addConstrs((f[v,i,j] <= x[i,j]   
                  for v in range(n) for i in range(n) for j in range(n) 
                  if v != r), 
                 name='4c')
    
    m.setObjective(gp.quicksum(data[i,j]*x[i,j] for i in range(n) for j in range(n)),
                   GRB.MINIMIZE)
    return m


def run_f3(logfileName, params, datasets):
    for name,data in datasets.items():
        with gp.Env() as env:       
            with gp.Model(env=env) as model3:
                try:
                    m3 = get_f3(data)
                    m3.setParam("LogFile", f'logs/{logfileName}_{name}.log')
                    
                    for key, value in params.items():
                        m3.setParam(key, value)
                    m3.optimize()
                    print_results(m3)
                except gp.GurobiError as e:
                    print('Error code ' + str(e.errno) + ': ' + str(e))
                except AttributeError:
                    print('Encountered an attribute error')

# Testing formulations



In [8]:
# reading datasets first
datasets = read_data('data')

### Test formulation 0 on four instances

In [9]:
for name, data in datasets.items():
    with gp.Env() as env:
    # with gp.Env(f'logs/f0_{name}.log') as env:
        with gp.Model(env=env) as model0:
            try:
                m0 = get_f0(data)
                m0.setParam("LogFile", f'logs/f0_{name}.log')
                m0.optimize()
                print_results(m0)
            except gp.GurobiError as e:
                print('Error code ' + str(e.errno) + ': ' + str(e))
            except AttributeError:
                print('Encountered an attribute error')

Set parameter Username
Set parameter LicenseID to value 2634667
Academic license - for non-commercial use only - expires 2026-03-10
Set parameter Username
Set parameter LicenseID to value 2634667
Academic license - for non-commercial use only - expires 2026-03-10
Set parameter LogFile to value "logs/f0_bays29.log"
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 87 rows, 841 columns and 1711 nonzeros
Model fingerprint: 0x4cfafabf
Variable types: 0 continuous, 841 integer (841 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 6280.0000000
Presolve removed 29 rows and 29 columns
Presolve time: 0.00s
Presolved: 58 rows, 812 

Thread count was 20 (of 20 available processors)

Solution count 2: 1089 8467 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.089000000000e+03, best bound 1.089000000000e+03, gap 0.0000%
Optimal Solution:
x[0,1]: 1.0
x[1,9]: 1.0
x[2,3]: 1.0
x[3,2]: 1.0
x[4,13]: 1.0
x[5,6]: 1.0
x[6,5]: 1.0
x[7,8]: 1.0
x[8,7]: 1.0
x[9,0]: 1.0
x[10,11]: 1.0
x[11,12]: 1.0
x[12,10]: 1.0
x[13,4]: 1.0
x[14,15]: 1.0
x[15,14]: 1.0
x[16,17]: 1.0
x[17,16]: 1.0
x[18,19]: 1.0
x[19,18]: 1.0
x[20,21]: 1.0
x[21,20]: 1.0
x[22,23]: 1.0
x[23,22]: 1.0
x[24,25]: 1.0
x[25,24]: 1.0
x[26,34]: 1.0
x[27,28]: 1.0
x[28,36]: 1.0
x[29,30]: 1.0
x[30,29]: 1.0
x[31,40]: 1.0
x[32,41]: 1.0
x[33,42]: 1.0
x[34,26]: 1.0
x[35,43]: 1.0
x[36,27]: 1.0
x[37,47]: 1.0
x[38,39]: 1.0
x[39,38]: 1.0
x[40,31]: 1.0
x[41,32]: 1.0
x[42,33]: 1.0
x[43,35]: 1.0
x[44,53]: 1.0
x[45,46]: 1.0
x[46,45]: 1.0
x[47,37]: 1.0
x[48,59]: 1.0
x[49,48]: 1.0
x[50,49]: 1.0
x[51,50]: 1.0
x[52,51]: 1.0
x[53,44]: 1.0
x[54,63]: 1.0
x[55,56]: 1.0
x[56,55]: 1.0
x[

### Test formulation 2 and 3 on four instances



#### a) with default settings: 

In [10]:
# FORMULATION 2
params = {'MIPGap':0, 'TimeLimit':3600}
run_f2('3a_f2', params, datasets)

Set parameter Username
Set parameter LicenseID to value 2634667
Academic license - for non-commercial use only - expires 2026-03-10
Set parameter LogFile to value "logs/3a_f2_bays29.log"
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 928 rows, 870 columns and 4064 nonzeros
Model fingerprint: 0xe20bd28c
Variable types: 0 continuous, 870 integer (841 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [3e+01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 114 rows and 30 columns
Presolve time: 0.00s
Presolved: 814 rows, 840 columns, 3892 nonzeros
Variable types: 0 continuous, 840 integer (812 binary)
Found heuristic solution: o

     0     0  637.50000    0   97  718.00000  637.50000  11.2%     -    0s
     0     0  637.50000    0   98  718.00000  637.50000  11.2%     -    0s
     0     0  638.00000    0  106  718.00000  638.00000  11.1%     -    0s
     0     0  638.00000    0  107  718.00000  638.00000  11.1%     -    0s
     0     0  638.00000    0   99  718.00000  638.00000  11.1%     -    0s
H    0     0                     713.0000000  638.00000  10.5%     -    0s
     0     0  638.00000    0  103  713.00000  638.00000  10.5%     -    0s
     0     0  638.00000    0   93  713.00000  638.00000  10.5%     -    0s
     0     2  638.00000    0   85  713.00000  638.00000  10.5%     -    0s
H   28    40                     710.0000000  638.20306  10.1%  30.2    0s
H   87   100                     704.0000000  638.65094  9.28%  14.7    0s
* 3381  1709              42     701.0000000  681.66641  2.76%  12.0    1s
* 3675  1533              36     700.0000000  682.52317  2.50%  12.3    2s
H 3739  1434             

H44332 15133                    109404.00000 106547.108  2.61%  25.1   81s
 47409 16508 106548.405   76  133 109404.000 106547.108  2.61%  25.3   85s
 52030 18073 109214.312  102  166 109404.000 106547.108  2.61%  25.3   90s
 56005 19626 108238.011  106   60 109404.000 106547.108  2.61%  25.3   96s
 59071 20953 107685.500   79   65 109404.000 106555.955  2.60%  25.3  100s
 64158 22459 108172.111  169  107 109404.000 106584.250  2.58%  25.5  105s
*66968 22103              92    109386.00000 106596.667  2.55%  25.7  108s
 69042 22672 108414.200  100   33 109386.000 106604.809  2.54%  25.8  111s
 73922 24353 106876.345   82  191 109386.000 106644.614  2.51%  26.1  116s
*75660 18176              94    108540.00000 106654.338  1.74%  26.0  117s
 78176 18354     cutoff  102      108540.000 106682.344  1.71%  26.3  120s
 82687 19535 107444.775   77  149 108540.000 106739.525  1.66%  26.8  125s
H87389 18631                    108343.00000 106798.745  1.43%  27.3  129s
 87392 18624 108191.478  

H  181   193                    1233.0000000 1203.00000  2.43%  29.8    5s
H  182   193                    1216.0000000 1203.00000  1.07%  29.7    5s
H  186   193                    1214.0000000 1203.00000  0.91%  29.2    5s
H  187   193                    1212.0000000 1203.00000  0.74%  29.0    5s
  1906   652 infeasible   15      1212.00000 1203.32895  0.72%  23.8   10s
* 4072   587              16    1211.0000000 1205.78912  0.43%  25.6   13s
  4818   244     cutoff   12      1211.00000 1207.02806  0.33%  26.3   15s

Cutting planes:
  Learned: 21
  Gomory: 21
  Lift-and-project: 2
  Cover: 10
  Implied bound: 156
  MIR: 157
  StrongCG: 6
  Inf proof: 48
  Zero half: 2
  Mod-K: 1
  RLT: 21
  Relax-and-lift: 89
  BQP: 11
  PSD: 13

Explored 5293 nodes (138783 simplex iterations) in 15.41 seconds (11.25 work units)
Thread count was 20 (of 20 available processors)

Solution count 10: 1211 1212 1214 ... 6074

Optimal solution found (tolerance 0.00e+00)
Best objective 1.211000000000e+03, 

In [11]:
# FORMULATION 3
params = {'MIPGap':0, 'TimeLimit':3600}
run_f3('3a_f3', params, datasets)

Set parameter Username
Set parameter LicenseID to value 2634667
Academic license - for non-commercial use only - expires 2026-03-10
Set parameter LogFile to value "logs/3a_f3_bays29.log"
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 24419 rows, 25230 columns and 92711 nonzeros
Model fingerprint: 0x1712e753
Variable types: 24389 continuous, 841 integer (841 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1625 rows and 2466 columns
Presolve time: 0.24s
Presolved: 22794 rows, 22764 columns, 88648 nonzeros
Variable types: 21952 continuous, 812 integer (812 binary)

Root 

Set parameter LicenseID to value 2634667
Academic license - for non-commercial use only - expires 2026-03-10
Set parameter LogFile to value "logs/3a_f3_dantzig42.log"
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 74131 rows, 75852 columns and 286060 nonzeros
Model fingerprint: 0x2b3b1d3e
Variable types: 74088 continuous, 1764 integer (1764 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 3445 rows and 5209 columns
Presolve time: 1.01s
Presolved: 70686 rows, 70643 columns, 277447 nonzeros
Variable types: 68921 continuous, 1722 integer (1722 binary)
Deterministic concur

Set parameter LicenseID to value 2634667
Academic license - for non-commercial use only - expires 2026-03-10
Set parameter LogFile to value "logs/3a_f3_pr76.log"
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 439053 rows, 444752 columns and 1721778 nonzeros
Model fingerprint: 0xe3d5fa2a
Variable types: 438976 continuous, 5776 integer (5776 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+02, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 5776 rows and 11552 columns (presolve time = 5s) ...
Presolve removed 5776 rows and 11552 columns
Presolve time: 9.29s
Presolved: 433277 rows, 433200 columns, 1710150 nonzeros
Variable

  2819   831 107606.000   18   46 108159.000 106805.000  1.25%  1912  809s
  3005   890     cutoff   12      108159.000 106837.167  1.22%  1876  823s
  3222   915 107268.667   13   81 108159.000 106875.417  1.19%  1836  838s
  3413   926     cutoff   15      108159.000 106891.000  1.17%  1817  861s
  3443   939 infeasible   16      108159.000 106912.000  1.15%  1815  875s
  3685   963 108004.500   13   61 108159.000 106925.500  1.14%  1774  889s
  3914   986     cutoff    9      108159.000 107016.500  1.06%  1743  903s
  4173  1018 107780.000   14   44 108159.000 107057.000  1.02%  1707  917s
  4430  1056     cutoff   20      108159.000 107084.000  0.99%  1678  932s
  4729  1083 107469.500   18   31 108159.000 107107.500  0.97%  1640  952s
  5060  1090     cutoff   16      108159.000 107141.000  0.94%  1599  970s
  5413  1107 108071.000   19   36 108159.000 107184.500  0.90%  1560  987s
  5734  1085 107991.000   19   32 108159.000 107200.500  0.89%  1537 1007s
  6130  1067 107749.500  

f[13,40,59]: 1.0
f[13,41,42]: 1.0
f[13,42,27]: 1.0
f[13,43,47]: 1.0
f[13,44,43]: 1.0
f[13,45,44]: 1.0
f[13,46,68]: 1.0
f[13,47,46]: 1.0
f[13,48,50]: 1.0
f[13,49,48]: 1.0
f[13,50,65]: 1.0
f[13,51,52]: 1.0
f[13,52,53]: 1.0
f[13,53,41]: 1.0
f[13,54,51]: 1.0
f[13,55,54]: 1.0
f[13,56,55]: 1.0
f[13,57,56]: 1.0
f[13,58,57]: 1.0
f[13,59,58]: 1.0
f[13,60,40]: 1.0
f[13,61,60]: 1.0
f[13,62,61]: 1.0
f[13,63,62]: 1.0
f[13,64,70]: 1.0
f[13,65,64]: 1.0
f[13,66,49]: 1.0
f[13,67,69]: 1.0
f[13,68,67]: 1.0
f[13,69,66]: 1.0
f[13,70,71]: 1.0
f[13,71,72]: 1.0
f[13,72,63]: 1.0
f[13,73,13]: 1.0
f[13,74,75]: 1.0
f[13,75,0]: 1.0
f[14,0,22]: 1.0
f[14,1,74]: 1.0
f[14,2,1]: 1.0
f[14,3,2]: 1.0
f[14,4,3]: 1.0
f[14,5,4]: 1.0
f[14,15,14]: 1.0
f[14,16,15]: 1.0
f[14,17,16]: 1.0
f[14,18,30]: 1.0
f[14,19,18]: 1.0
f[14,20,24]: 1.0
f[14,21,20]: 1.0
f[14,22,21]: 1.0
f[14,23,45]: 1.0
f[14,24,23]: 1.0
f[14,25,19]: 1.0
f[14,26,25]: 1.0
f[14,27,26]: 1.0
f[14,28,31]: 1.0
f[14,29,28]: 1.0
f[14,30,29]: 1.0
f[14,31,32]: 1.0
f[14,32,

...

f[32,18,27]: 1.0
f[32,19,18]: 1.0
f[32,20,19]: 1.0
f[32,21,20]: 1.0
f[32,27,36]: 1.0
f[32,28,29]: 1.0
f[32,29,30]: 1.0
f[32,30,31]: 1.0
f[32,31,32]: 1.0
f[32,36,28]: 1.0
f[33,0,9]: 1.0
f[33,1,0]: 1.0
f[33,2,1]: 1.0
f[33,3,2]: 1.0
f[33,4,13]: 1.0
f[33,5,4]: 1.0
f[33,9,10]: 1.0
f[33,10,11]: 1.0
f[33,11,12]: 1.0
f[33,12,21]: 1.0
f[33,13,3]: 1.0
f[33,18,27]: 1.0
f[33,19,18]: 1.0
f[33,20,19]: 1.0
f[33,21,20]: 1.0
f[33,27,36]: 1.0
f[33,28,29]: 1.0
f[33,29,30]: 1.0
f[33,30,31]: 1.0
f[33,31,32]: 1.0
f[33,32,33]: 1.0
f[33,36,28]: 1.0
f[34,0,9]: 1.0
f[34,1,0]: 1.0
f[34,2,1]: 1.0
f[34,3,2]: 1.0
f[34,4,13]: 1.0
f[34,5,4]: 1.0
f[34,9,10]: 1.0
f[34,10,11]: 1.0
f[34,11,12]: 1.0
f[34,12,21]: 1.0
f[34,13,3]: 1.0
f[34,18,27]: 1.0
f[34,19,18]: 1.0
f[34,20,19]: 1.0
f[34,21,20]: 1.0
f[34,27,36]: 1.0
f[34,28,29]: 1.0
f[34,29,30]: 1.0
f[34,30,31]: 1.0
f[34,31,32]: 1.0
f[34,32,33]: 1.0
f[34,33,42]: 1.0
f[34,35,34]: 1.0
f[34,36,28]: 1.0
f[34,37,46]: 1.0
f[34,38,47]: 1.0
f[34,39,38]: 1.0
f[34,40,39]: 1.0
f[34,4

f[55,0,9]: 1.0
f[55,1,0]: 1.0
f[55,2,1]: 1.0
f[55,3,2]: 1.0
f[55,4,13]: 1.0
f[55,5,4]: 1.0
f[55,9,10]: 1.0
f[55,10,11]: 1.0
f[55,11,12]: 1.0
f[55,12,21]: 1.0
f[55,13,3]: 1.0
f[55,18,27]: 1.0
f[55,19,18]: 1.0
f[55,20,19]: 1.0
f[55,21,20]: 1.0
f[55,27,36]: 1.0
f[55,28,29]: 1.0
f[55,29,30]: 1.0
f[55,30,31]: 1.0
f[55,31,32]: 1.0
f[55,32,33]: 1.0
f[55,33,42]: 1.0
f[55,36,28]: 1.0
f[55,37,46]: 1.0
f[55,38,47]: 1.0
f[55,39,38]: 1.0
f[55,40,39]: 1.0
f[55,41,40]: 1.0
f[55,42,41]: 1.0
f[55,45,54]: 1.0
f[55,46,45]: 1.0
f[55,47,37]: 1.0
f[55,54,63]: 1.0
f[55,63,64]: 1.0
f[55,64,55]: 1.0
f[56,0,9]: 1.0
f[56,1,0]: 1.0
f[56,2,1]: 1.0
f[56,3,2]: 1.0
f[56,4,13]: 1.0
f[56,5,4]: 1.0
f[56,9,10]: 1.0
f[56,10,11]: 1.0
f[56,11,12]: 1.0
f[56,12,21]: 1.0
f[56,13,3]: 1.0
f[56,18,27]: 1.0
f[56,19,18]: 1.0
f[56,20,19]: 1.0
f[56,21,20]: 1.0
f[56,27,36]: 1.0
f[56,28,29]: 1.0
f[56,29,30]: 1.0
f[56,30,31]: 1.0
f[56,31,32]: 1.0
f[56,32,33]: 1.0
f[56,33,42]: 1.0
f[56,36,28]: 1.0
f[56,37,46]: 1.0
f[56,38,47]: 1.0
f[56,3

f[67,0,9]: 1.0
f[67,1,0]: 1.0
f[67,2,1]: 1.0
f[67,3,2]: 1.0
f[67,4,13]: 1.0
f[67,5,4]: 1.0
f[67,9,10]: 1.0
f[67,10,11]: 1.0
f[67,11,12]: 1.0
f[67,12,21]: 1.0
f[67,13,3]: 1.0
f[67,18,27]: 1.0
f[67,19,18]: 1.0
f[67,20,19]: 1.0
f[67,21,20]: 1.0
f[67,27,36]: 1.0
f[67,28,29]: 1.0
f[67,29,30]: 1.0
f[67,30,31]: 1.0
f[67,31,32]: 1.0
f[67,32,33]: 1.0
f[67,33,42]: 1.0
f[67,36,28]: 1.0
f[67,37,46]: 1.0
f[67,38,47]: 1.0
f[67,39,38]: 1.0
f[67,40,39]: 1.0
f[67,41,40]: 1.0
f[67,42,41]: 1.0
f[67,45,54]: 1.0
f[67,46,45]: 1.0
f[67,47,37]: 1.0
f[67,54,63]: 1.0
f[67,55,56]: 1.0
f[67,56,57]: 1.0
f[67,57,58]: 1.0
f[67,58,67]: 1.0
f[67,63,64]: 1.0
f[67,64,55]: 1.0
f[68,0,9]: 1.0
f[68,1,0]: 1.0
f[68,2,1]: 1.0
f[68,3,2]: 1.0
f[68,4,13]: 1.0
f[68,5,4]: 1.0
f[68,9,10]: 1.0
f[68,10,11]: 1.0
f[68,11,12]: 1.0
f[68,12,21]: 1.0
f[68,13,3]: 1.0
f[68,18,27]: 1.0
f[68,19,18]: 1.0
f[68,20,19]: 1.0
f[68,21,20]: 1.0
f[68,27,36]: 1.0
f[68,28,29]: 1.0
f[68,29,30]: 1.0
f[68,30,31]: 1.0
f[68,31,32]: 1.0
f[68,32,33]: 1.0
f[68,3

f[78,88,78]: 1.0
f[78,89,88]: 1.0
f[78,90,91]: 1.0
f[78,91,92]: 1.0
f[78,92,93]: 1.0
f[78,93,94]: 1.0
f[78,94,95]: 1.0
f[78,95,96]: 1.0
f[78,96,97]: 1.0
f[78,97,98]: 1.0
f[78,98,89]: 1.0
f[79,0,9]: 1.0
f[79,1,0]: 1.0
f[79,2,1]: 1.0
f[79,3,2]: 1.0
f[79,4,13]: 1.0
f[79,5,4]: 1.0
f[79,9,10]: 1.0
f[79,10,11]: 1.0
f[79,11,12]: 1.0
f[79,12,21]: 1.0
f[79,13,3]: 1.0
f[79,18,27]: 1.0
f[79,19,18]: 1.0
f[79,20,19]: 1.0
f[79,21,20]: 1.0
f[79,27,36]: 1.0
f[79,28,29]: 1.0
f[79,29,30]: 1.0
f[79,30,31]: 1.0
f[79,31,32]: 1.0
f[79,32,33]: 1.0
f[79,33,42]: 1.0
f[79,36,28]: 1.0
f[79,37,46]: 1.0
f[79,38,47]: 1.0
f[79,39,38]: 1.0
f[79,40,39]: 1.0
f[79,41,40]: 1.0
f[79,42,41]: 1.0
f[79,45,54]: 1.0
f[79,46,45]: 1.0
f[79,47,37]: 1.0
f[79,54,63]: 1.0
f[79,55,56]: 1.0
f[79,56,57]: 1.0
f[79,57,58]: 1.0
f[79,58,67]: 1.0
f[79,63,64]: 1.0
f[79,64,55]: 1.0
f[79,65,74]: 1.0
f[79,66,65]: 1.0
f[79,67,66]: 1.0
f[79,68,69]: 1.0
f[79,69,70]: 1.0
f[79,70,79]: 1.0
f[79,72,81]: 1.0
f[79,73,72]: 1.0
f[79,74,73]: 1.0
f[79,75,76

f[87,63,64]: 1.0
f[87,64,55]: 1.0
f[87,65,74]: 1.0
f[87,66,65]: 1.0
f[87,67,66]: 1.0
f[87,72,81]: 1.0
f[87,73,72]: 1.0
f[87,74,73]: 1.0
f[87,78,87]: 1.0
f[87,81,82]: 1.0
f[87,82,90]: 1.0
f[87,88,78]: 1.0
f[87,89,88]: 1.0
f[87,90,91]: 1.0
f[87,91,92]: 1.0
f[87,92,93]: 1.0
f[87,93,94]: 1.0
f[87,94,95]: 1.0
f[87,95,96]: 1.0
f[87,96,97]: 1.0
f[87,97,98]: 1.0
f[87,98,89]: 1.0
f[88,0,9]: 1.0
f[88,1,0]: 1.0
f[88,2,1]: 1.0
f[88,3,2]: 1.0
f[88,4,13]: 1.0
f[88,5,4]: 1.0
f[88,9,10]: 1.0
f[88,10,11]: 1.0
f[88,11,12]: 1.0
f[88,12,21]: 1.0
f[88,13,3]: 1.0
f[88,18,27]: 1.0
f[88,19,18]: 1.0
f[88,20,19]: 1.0
f[88,21,20]: 1.0
f[88,27,36]: 1.0
f[88,28,29]: 1.0
f[88,29,30]: 1.0
f[88,30,31]: 1.0
f[88,31,32]: 1.0
f[88,32,33]: 1.0
f[88,33,42]: 1.0
f[88,36,28]: 1.0
f[88,37,46]: 1.0
f[88,38,47]: 1.0
f[88,39,38]: 1.0
f[88,40,39]: 1.0
f[88,41,40]: 1.0
f[88,42,41]: 1.0
f[88,45,54]: 1.0
f[88,46,45]: 1.0
f[88,47,37]: 1.0
f[88,54,63]: 1.0
f[88,55,56]: 1.0
f[88,56,57]: 1.0
f[88,57,58]: 1.0
f[88,58,67]: 1.0
f[88,63,64

#### b) with `presolve` truned off: 

In [12]:
# FORMULATION 2
params = {'MIPGap':0, 'TimeLimit':3600, 'Presolve':0}
run_f2('3b_f2', params, datasets)

Set parameter Username
Set parameter LicenseID to value 2634667
Academic license - for non-commercial use only - expires 2026-03-10
Set parameter LogFile to value "logs/3b_f2_bays29.log"
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 3600
Set parameter Presolve to value 0
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 928 rows, 870 columns and 4064 nonzeros
Model fingerprint: 0xe20bd28c
Variable types: 0 continuous, 870 integer (841 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [3e+01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Variable types: 0 continuous, 870 integer (841 binary)

Root relaxation: objective 1.774000e+03, 118 iterations, 0.00 seconds (0.00 work units)

    Nodes    

  Learned: 390
  Gomory: 133
  Lift-and-project: 8
  Cover: 21
  Implied bound: 9
  Projected implied bound: 16
  MIR: 34
  Mixing: 5
  StrongCG: 1
  Flow cover: 165
  Inf proof: 20
  Zero half: 21
  Mod-K: 3
  RLT: 29
  Relax-and-lift: 31
  BQP: 3
  PSD: 3

Explored 22621 nodes (341522 simplex iterations) in 24.18 seconds (8.37 work units)
Thread count was 20 (of 20 available processors)

Solution count 10: 699 705 706 ... 735

Optimal solution found (tolerance 0.00e+00)
Best objective 6.990000000000e+02, best bound 6.990000000000e+02, gap 0.0000%
Optimal Solution:
x[0,1]: 1.0
x[1,2]: 1.0
x[2,3]: 1.0
x[3,4]: 1.0
x[4,5]: 1.0
x[5,6]: 1.0
x[6,7]: 1.0
x[7,8]: 1.0
x[8,9]: 1.0
x[9,10]: 1.0
x[10,11]: 1.0
x[11,12]: 1.0
x[12,13]: 1.0
x[13,14]: 1.0
x[14,15]: 1.0
x[15,16]: 1.0
x[16,17]: 1.0
x[17,18]: 1.0
x[18,19]: 1.0
x[19,20]: 1.0
x[20,21]: 1.0
x[21,22]: 1.0
x[22,23]: 1.0
x[23,24]: 1.0
x[24,25]: 1.0
x[25,26]: 1.0
x[26,27]: 1.0
x[27,28]: 1.0
x[28,29]: 1.0
x[29,30]: 1.0
x[30,31]: 1.0
x[31,32]: 1.

 45547 12202 infeasible   29      108183.000 106730.793  1.34%  29.3  239s
 46790 12250 107613.667   30   89 108183.000 106756.819  1.32%  29.4  244s
 48029 12352 107554.677   41   94 108183.000 106786.769  1.29%  29.5  249s
 49406 12389 108145.361   28  196 108183.000 106818.294  1.26%  29.6  253s
 50657 12390 107956.237   37   74 108183.000 106845.886  1.24%  29.8  258s
 51909 12462     cutoff   40      108183.000 106866.083  1.22%  29.9  264s
 53302 12560 107410.140   31  149 108183.000 106886.139  1.20%  30.0  268s
 54582 12568 107214.899   37   74 108183.000 106911.140  1.18%  30.2  272s
 55651 12664 infeasible   40      108183.000 106933.450  1.16%  30.3  278s
 57043 12665     cutoff   42      108183.000 106959.169  1.13%  30.4  283s
 58299 12629 infeasible   32      108183.000 106984.168  1.11%  30.5  288s
 59681 12650 107660.333   35  125 108183.000 107018.133  1.08%  30.6  294s
 61085 12621 108114.250   37   23 108183.000 107042.699  1.05%  30.7  299s
H62290 12524             

     0     0 1198.50000    0  266          - 1198.50000      -     -    2s
     0     0 1198.50000    0  264          - 1198.50000      -     -    3s
     0     0 1198.50000    0  266          - 1198.50000      -     -    3s
H    0     0                    1454.0000000 1198.50000  17.6%     -    3s
     0     0 1198.50000    0  174 1454.00000 1198.50000  17.6%     -    3s
H    0     0                    1289.0000000 1198.50000  7.02%     -    5s
     0     0 1198.50000    0  171 1289.00000 1198.50000  7.02%     -    5s
     0     0 1198.50000    0  213 1289.00000 1198.50000  7.02%     -    6s
     0     0 1198.50000    0  248 1289.00000 1198.50000  7.02%     -    6s
     0     0 1198.50000    0  200 1289.00000 1198.50000  7.02%     -    6s
     0     0 1198.50000    0  200 1289.00000 1198.50000  7.02%     -    6s
     0     0 1198.70000    0  206 1289.00000 1198.70000  7.01%     -    7s
     0     0 1199.00000    0  205 1289.00000 1199.00000  6.98%     -    7s
     0     0 1199.00000  

In [13]:
# FORMULATION 3
params = {'MIPGap':0, 'TimeLimit':3600, 'Presolve':0}
run_f3('3b_f3', params, datasets)

Set parameter Username
Set parameter LicenseID to value 2634667
Academic license - for non-commercial use only - expires 2026-03-10
Set parameter LogFile to value "logs/3b_f3_bays29.log"
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 3600
Set parameter Presolve to value 0
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 24419 rows, 25230 columns and 92711 nonzeros
Model fingerprint: 0x1712e753
Variable types: 24389 continuous, 841 integer (841 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Variable types: 24389 continuous, 841 integer (841 binary)

Root relaxation: objective 2.013500e+03, 16234 iterations, 1.92 seconds (1.96 work units)

Set parameter LicenseID to value 2634667
Academic license - for non-commercial use only - expires 2026-03-10
Set parameter LogFile to value "logs/3b_f3_dantzig42.log"
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 3600
Set parameter Presolve to value 0
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 74131 rows, 75852 columns and 286060 nonzeros
Model fingerprint: 0x2b3b1d3e
Variable types: 74088 continuous, 1764 integer (1764 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Variable types: 74088 continuous, 1764 integer (1764 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing primal log only...


Root simplex

f[27,24,25]: 1.0
f[27,25,26]: 1.0
f[27,26,27]: 1.0
f[28,5,6]: 1.0
f[28,6,7]: 1.0
f[28,7,8]: 1.0
f[28,8,9]: 1.0
f[28,9,10]: 1.0
f[28,10,11]: 1.0
f[28,11,12]: 1.0
f[28,12,13]: 1.0
f[28,13,14]: 1.0
f[28,14,15]: 1.0
f[28,15,16]: 1.0
f[28,16,17]: 1.0
f[28,17,18]: 1.0
f[28,18,19]: 1.0
f[28,19,20]: 1.0
f[28,20,21]: 1.0
f[28,21,22]: 1.0
f[28,22,23]: 1.0
f[28,23,24]: 1.0
f[28,24,25]: 1.0
f[28,25,26]: 1.0
f[28,26,27]: 1.0
f[28,27,28]: 1.0
f[29,5,6]: 1.0
f[29,6,7]: 1.0
f[29,7,8]: 1.0
f[29,8,9]: 1.0
f[29,9,10]: 1.0
f[29,10,11]: 1.0
f[29,11,12]: 1.0
f[29,12,13]: 1.0
f[29,13,14]: 1.0
f[29,14,15]: 1.0
f[29,15,16]: 1.0
f[29,16,17]: 1.0
f[29,17,18]: 1.0
f[29,18,19]: 1.0
f[29,19,20]: 1.0
f[29,20,21]: 1.0
f[29,21,22]: 1.0
f[29,22,23]: 1.0
f[29,23,24]: 1.0
f[29,24,25]: 1.0
f[29,25,26]: 1.0
f[29,26,27]: 1.0
f[29,27,28]: 1.0
f[29,28,29]: 1.0
f[30,5,6]: 1.0
f[30,6,7]: 1.0
f[30,7,8]: 1.0
f[30,8,9]: 1.0
f[30,9,10]: 1.0
f[30,10,11]: 1.0
f[30,11,12]: 1.0
f[30,12,13]: 1.0
f[30,13,14]: 1.0
f[30,14,15]: 1.0
f[30,15

 Threads    : 10

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   3.82913782e+07 -2.22975812e+08  6.65e+01 0.00e+00  5.91e+02    26s
   1   2.41478491e+06 -4.17790158e+07  3.29e+00 3.18e-11  4.91e+01    28s
   2   8.39250924e+05 -2.41288896e+06  5.19e-01 8.45e-11  4.27e+00    32s
   3   5.30680597e+05 -5.70534622e+05  1.70e-02 3.82e-11  8.58e-01    36s
   4   4.35103897e+05 -9.69524798e+04  2.75e-03 3.96e-11  4.06e-01    41s
   5   4.00456762e+05 -9.92093918e+03  2.23e-03 4.77e-11  3.13e-01    45s
   6   3.62580414e+05  2.72637242e+04  1.78e-03 5.37e-11  2.56e-01    48s
   7   2.93824502e+05  4.19847916e+04  1.14e-03 4.25e-11  1.92e-01    51s
   8   2.59821387e+05  5.61377170e+04  9.00e-04 5.34e-11  1.55e-01    55s
   9   2.24423725e+05  6.98451216e+04  6.79e-04 6.03e-11  1.18e-01    59s
  10   1.89856135e+05  8.05971449e+04  4.55e-04 6.69e-11  8.33e-02    62s
  11   1.64337674e+05  9.06198301e+04  3.0

     0     0 1206.00000    0   85          - 1206.00000      -     -  479s
     0     0 1206.00000    0   85          - 1206.00000      -     -  534s
H    0     0                    1211.0000000 1206.00000  0.41%     -  690s
     0     2 1206.00000    0   85 1211.00000 1206.00000  0.41%     -  724s
     1     3 1206.00000    1   69 1211.00000 1206.00000  0.41%   890  750s
     3     5 1206.00000    2   85 1211.00000 1206.00000  0.41%  2140  793s
     6     4 1208.00000    3   95 1211.00000 1208.00000  0.25%  2445  829s
    11     3 1208.00000    4  202 1211.00000 1208.00000  0.25%  2061  938s
    15     1 1208.31250    5  205 1211.00000 1208.31250  0.22%  2025  958s
    18     1 1208.31250    6  205 1211.00000 1208.31250  0.22%  1734  976s
    19     1 1208.31250    7  190 1211.00000 1208.31250  0.22%  1729  991s
    20     1 1209.25000    8  150 1211.00000 1209.25000  0.14%  1712 1052s

Cutting planes:
  Gomory: 1
  Lift-and-project: 2

Explored 22 nodes (510922 simplex iterations) in

...

f[45,50,49]: 1.0
f[45,51,50]: 1.0
f[45,52,51]: 1.0
f[45,53,52]: 1.0
f[45,54,45]: 1.0
f[45,55,64]: 1.0
f[45,56,55]: 1.0
f[45,57,56]: 1.0
f[45,58,57]: 1.0
f[45,59,60]: 1.0
f[45,60,61]: 1.0
f[45,61,62]: 1.0
f[45,62,71]: 1.0
f[45,63,54]: 1.0
f[45,64,63]: 1.0
f[45,65,66]: 1.0
f[45,66,67]: 1.0
f[45,67,58]: 1.0
f[45,68,77]: 1.0
f[45,69,68]: 1.0
f[45,70,69]: 1.0
f[45,71,80]: 1.0
f[45,72,73]: 1.0
f[45,73,74]: 1.0
f[45,74,65]: 1.0
f[45,75,83]: 1.0
f[45,76,75]: 1.0
f[45,77,76]: 1.0
f[45,78,88]: 1.0
f[45,79,70]: 1.0
f[45,80,79]: 1.0
f[45,81,72]: 1.0
f[45,82,81]: 1.0
f[45,83,84]: 1.0
f[45,84,85]: 1.0
f[45,85,86]: 1.0
f[45,86,87]: 1.0
f[45,87,78]: 1.0
f[45,88,89]: 1.0
f[45,89,98]: 1.0
f[45,90,82]: 1.0
f[45,91,90]: 1.0
f[45,92,91]: 1.0
f[45,93,92]: 1.0
f[45,94,93]: 1.0
f[45,95,94]: 1.0
f[45,96,95]: 1.0
f[45,97,96]: 1.0
f[45,98,97]: 1.0
f[46,5,6]: 1.0
f[46,6,7]: 1.0
f[46,7,8]: 1.0
f[46,8,17]: 1.0
f[46,14,22]: 1.0
f[46,15,14]: 1.0
f[46,16,15]: 1.0
f[46,17,16]: 1.0
f[46,22,23]: 1.0
f[46,23,24]: 1.0
f[46

f[58,22,23]: 1.0
f[58,23,24]: 1.0
f[58,24,25]: 1.0
f[58,25,26]: 1.0
f[58,26,34]: 1.0
f[58,34,35]: 1.0
f[58,35,43]: 1.0
f[58,43,44]: 1.0
f[58,44,53]: 1.0
f[58,48,59]: 1.0
f[58,49,48]: 1.0
f[58,50,49]: 1.0
f[58,51,50]: 1.0
f[58,52,51]: 1.0
f[58,53,52]: 1.0
f[58,59,60]: 1.0
f[58,60,61]: 1.0
f[58,61,62]: 1.0
f[58,62,71]: 1.0
f[58,65,66]: 1.0
f[58,66,67]: 1.0
f[58,67,58]: 1.0
f[58,68,77]: 1.0
f[58,69,68]: 1.0
f[58,70,69]: 1.0
f[58,71,80]: 1.0
f[58,72,73]: 1.0
f[58,73,74]: 1.0
f[58,74,65]: 1.0
f[58,75,83]: 1.0
f[58,76,75]: 1.0
f[58,77,76]: 1.0
f[58,78,88]: 1.0
f[58,79,70]: 1.0
f[58,80,79]: 1.0
f[58,81,72]: 1.0
f[58,82,81]: 1.0
f[58,83,84]: 1.0
f[58,84,85]: 1.0
f[58,85,86]: 1.0
f[58,86,87]: 1.0
f[58,87,78]: 1.0
f[58,88,89]: 1.0
f[58,89,98]: 1.0
f[58,90,82]: 1.0
f[58,91,90]: 1.0
f[58,92,91]: 1.0
f[58,93,92]: 1.0
f[58,94,93]: 1.0
f[58,95,94]: 1.0
f[58,96,95]: 1.0
f[58,97,96]: 1.0
f[58,98,97]: 1.0
f[59,5,6]: 1.0
f[59,6,7]: 1.0
f[59,7,8]: 1.0
f[59,8,17]: 1.0
f[59,14,22]: 1.0
f[59,15,14]: 1.0
f[59

f[70,48,59]: 1.0
f[70,49,48]: 1.0
f[70,50,49]: 1.0
f[70,51,50]: 1.0
f[70,52,51]: 1.0
f[70,53,52]: 1.0
f[70,59,60]: 1.0
f[70,60,61]: 1.0
f[70,61,62]: 1.0
f[70,62,71]: 1.0
f[70,71,80]: 1.0
f[70,79,70]: 1.0
f[70,80,79]: 1.0
f[71,5,6]: 1.0
f[71,6,7]: 1.0
f[71,7,8]: 1.0
f[71,8,17]: 1.0
f[71,14,22]: 1.0
f[71,15,14]: 1.0
f[71,16,15]: 1.0
f[71,17,16]: 1.0
f[71,22,23]: 1.0
f[71,23,24]: 1.0
f[71,24,25]: 1.0
f[71,25,26]: 1.0
f[71,26,34]: 1.0
f[71,34,35]: 1.0
f[71,35,43]: 1.0
f[71,43,44]: 1.0
f[71,44,53]: 1.0
f[71,48,59]: 1.0
f[71,49,48]: 1.0
f[71,50,49]: 1.0
f[71,51,50]: 1.0
f[71,52,51]: 1.0
f[71,53,52]: 1.0
f[71,59,60]: 1.0
f[71,60,61]: 1.0
f[71,61,62]: 1.0
f[71,62,71]: 1.0
f[72,5,6]: 1.0
f[72,6,7]: 1.0
f[72,7,8]: 1.0
f[72,8,17]: 1.0
f[72,14,22]: 1.0
f[72,15,14]: 1.0
f[72,16,15]: 1.0
f[72,17,16]: 1.0
f[72,22,23]: 1.0
f[72,23,24]: 1.0
f[72,24,25]: 1.0
f[72,25,26]: 1.0
f[72,26,34]: 1.0
f[72,34,35]: 1.0
f[72,35,43]: 1.0
f[72,43,44]: 1.0
f[72,44,53]: 1.0
f[72,48,59]: 1.0
f[72,49,48]: 1.0
f[72,50,49]

f[85,59,60]: 1.0
f[85,60,61]: 1.0
f[85,61,62]: 1.0
f[85,62,71]: 1.0
f[85,68,77]: 1.0
f[85,69,68]: 1.0
f[85,70,69]: 1.0
f[85,71,80]: 1.0
f[85,75,83]: 1.0
f[85,76,75]: 1.0
f[85,77,76]: 1.0
f[85,79,70]: 1.0
f[85,80,79]: 1.0
f[85,83,84]: 1.0
f[85,84,85]: 1.0
f[86,5,6]: 1.0
f[86,6,7]: 1.0
f[86,7,8]: 1.0
f[86,8,17]: 1.0
f[86,14,22]: 1.0
f[86,15,14]: 1.0
f[86,16,15]: 1.0
f[86,17,16]: 1.0
f[86,22,23]: 1.0
f[86,23,24]: 1.0
f[86,24,25]: 1.0
f[86,25,26]: 1.0
f[86,26,34]: 1.0
f[86,34,35]: 1.0
f[86,35,43]: 1.0
f[86,43,44]: 1.0
f[86,44,53]: 1.0
f[86,48,59]: 1.0
f[86,49,48]: 1.0
f[86,50,49]: 1.0
f[86,51,50]: 1.0
f[86,52,51]: 1.0
f[86,53,52]: 1.0
f[86,59,60]: 1.0
f[86,60,61]: 1.0
f[86,61,62]: 1.0
f[86,62,71]: 1.0
f[86,68,77]: 1.0
f[86,69,68]: 1.0
f[86,70,69]: 1.0
f[86,71,80]: 1.0
f[86,75,83]: 1.0
f[86,76,75]: 1.0
f[86,77,76]: 1.0
f[86,79,70]: 1.0
f[86,80,79]: 1.0
f[86,83,84]: 1.0
f[86,84,85]: 1.0
f[86,85,86]: 1.0
f[87,5,6]: 1.0
f[87,6,7]: 1.0
f[87,7,8]: 1.0
f[87,8,17]: 1.0
f[87,14,22]: 1.0
f[87,15,14]

f[96,71,80]: 1.0
f[96,75,83]: 1.0
f[96,76,75]: 1.0
f[96,77,76]: 1.0
f[96,78,88]: 1.0
f[96,79,70]: 1.0
f[96,80,79]: 1.0
f[96,83,84]: 1.0
f[96,84,85]: 1.0
f[96,85,86]: 1.0
f[96,86,87]: 1.0
f[96,87,78]: 1.0
f[96,88,89]: 1.0
f[96,89,98]: 1.0
f[96,97,96]: 1.0
f[96,98,97]: 1.0
f[97,5,6]: 1.0
f[97,6,7]: 1.0
f[97,7,8]: 1.0
f[97,8,17]: 1.0
f[97,14,22]: 1.0
f[97,15,14]: 1.0
f[97,16,15]: 1.0
f[97,17,16]: 1.0
f[97,22,23]: 1.0
f[97,23,24]: 1.0
f[97,24,25]: 1.0
f[97,25,26]: 1.0
f[97,26,34]: 1.0
f[97,34,35]: 1.0
f[97,35,43]: 1.0
f[97,43,44]: 1.0
f[97,44,53]: 1.0
f[97,48,59]: 1.0
f[97,49,48]: 1.0
f[97,50,49]: 1.0
f[97,51,50]: 1.0
f[97,52,51]: 1.0
f[97,53,52]: 1.0
f[97,59,60]: 1.0
f[97,60,61]: 1.0
f[97,61,62]: 1.0
f[97,62,71]: 1.0
f[97,68,77]: 1.0
f[97,69,68]: 1.0
f[97,70,69]: 1.0
f[97,71,80]: 1.0
f[97,75,83]: 1.0
f[97,76,75]: 1.0
f[97,77,76]: 1.0
f[97,78,88]: 1.0
f[97,79,70]: 1.0
f[97,80,79]: 1.0
f[97,83,84]: 1.0
f[97,84,85]: 1.0
f[97,85,86]: 1.0
f[97,86,87]: 1.0
f[97,87,78]: 1.0
f[97,88,89]: 1.0
f[97

#### c) with `cuts` turned off: 

In [14]:
# FORMULATION 2
params = {'MIPGap':0, 'TimeLimit':3600, 'Cuts':0}
run_f2('3c_f2', params, datasets)

Set parameter Username
Set parameter LicenseID to value 2634667
Academic license - for non-commercial use only - expires 2026-03-10
Set parameter LogFile to value "logs/3c_f2_bays29.log"
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 3600
Set parameter Cuts to value 0
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 928 rows, 870 columns and 4064 nonzeros
Model fingerprint: 0xe20bd28c
Variable types: 0 continuous, 870 integer (841 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [3e+01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 114 rows and 30 columns
Presolve time: 0.01s
Presolved: 814 rows, 840 columns, 3892 nonzeros
Variable types: 0 continuous, 840 integer (812 binar

 212141 31956  669.99634   32   56  699.00000  664.31667  4.96%  15.6  195s
 216257 30974     cutoff   34       699.00000  664.82500  4.89%  15.6  200s
 222356 29184 infeasible   36       699.00000  665.60000  4.78%  15.7  206s
 226338 27744     cutoff   30       699.00000  666.20000  4.69%  15.8  210s
 232407 25138 infeasible   31       699.00000  667.15000  4.56%  15.9  216s
 236369 23170     cutoff   39       699.00000  667.87500  4.45%  16.0  220s
 242472 19759     cutoff   46       699.00000  669.10970  4.28%  16.1  226s
 246570 17206 infeasible   32       699.00000  670.07857  4.14%  16.2  230s
 252748 12411 infeasible   42       699.00000  672.07500  3.85%  16.2  235s
 259258  5717 infeasible   29       699.00000  676.59583  3.21%  16.3  240s

Explored 267589 nodes (4306710 simplex iterations) in 243.55 seconds (35.18 work units)
Thread count was 20 (of 20 available processors)

Solution count 10: 699 739 741 ... 2817

Optimal solution found (tolerance 0.00e+00)
Best objective 6

 92635 51578 109575.595   45   64 110490.000 93356.5759  15.5%  19.3  226s
 95136 53337 99631.8036   28  103 110490.000 93402.5632  15.5%  19.4  233s
 96532 54230 95903.8233   27  125 110490.000 93410.8317  15.5%  19.4  236s
 99335 55740 97762.8604   24   89 110490.000 93463.7206  15.4%  19.4  241s
 101698 57119 103867.237   34   95 110490.000 93498.6771  15.4%  19.5  247s
 104009 58663 108064.351   40   81 110490.000 93521.1578  15.4%  19.5  252s
 105410 59409 93673.8081   28  125 110490.000 93535.1781  15.3%  19.5  255s
 107943 60891 107972.313   38  104 110490.000 93570.7494  15.3%  19.5  261s
 110881 62367 94912.2389   25  117 110490.000 93616.9777  15.3%  19.6  267s
 113453 63638     cutoff   38      110490.000 93648.3132  15.2%  19.6  272s
 115755 65016 108565.459   41  111 110490.000 93676.2774  15.2%  19.6  277s
 116954 65710 110193.338   38   81 110490.000 93687.9470  15.2%  19.7  280s
 119425 67078 96543.1578   27  109 110490.000 93710.5306  15.2%  19.7  285s
 122187 68515 10

 361053 190465 108523.428   38   98 110490.000 95200.6054  13.8%  21.2  772s
 363533 191798 105441.214   32  105 110490.000 95211.3018  13.8%  21.2  776s
 366237 193160 105518.146   36   96 110490.000 95219.9619  13.8%  21.2  782s
 367418 193712 107356.726   33   89 110490.000 95224.9461  13.8%  21.2  785s
 370258 194515 97837.2322   28  125 110490.000 95234.5283  13.8%  21.2  790s
 372948 195329 infeasible   45      110490.000 95244.8876  13.8%  21.2  795s
 375518 196708 100136.261   30  133 110490.000 95251.5768  13.8%  21.2  800s
 377930 197921 109126.824   38  103 110490.000 95257.9551  13.8%  21.2  805s
 380583 199188 99818.7748   30   96 110490.000 95269.9641  13.8%  21.2  810s
 383357 200313 infeasible   38      110490.000 95278.4299  13.8%  21.2  815s
 385954 201409 infeasible   47      110490.000 95295.4957  13.8%  21.2  821s
 388718 202665     cutoff   39      110490.000 95306.1511  13.7%  21.2  826s
 391258 203978     cutoff   36      110490.000 95319.8851  13.7%  21.2  831s

 568533 291035 108665.973   40   75 110490.000 95929.9822  13.2%  21.7 1333s
 569790 291667 102209.392   29  110 110490.000 95935.7727  13.2%  21.7 1338s
 571356 292360 96376.7743   28   91 110490.000 95940.2658  13.2%  21.7 1341s
 572527 293115 101874.745   29  109 110490.000 95942.6195  13.2%  21.7 1345s
 575151 294290 107774.730   37   64 110490.000 95948.5631  13.2%  21.7 1353s
 576445 295035 97136.1369   26  107 110490.000 95952.5811  13.2%  21.7 1358s
 577674 295618     cutoff   34      110490.000 95955.1114  13.2%  21.7 1362s
 579259 296132 103169.517   30  106 110490.000 95958.9191  13.2%  21.7 1366s
 580565 296787     cutoff   40      110490.000 95962.7333  13.1%  21.7 1370s
 583219 298203 109636.068   46  100 110490.000 95970.9611  13.1%  21.7 1378s
 584384 298947 109980.135   49   78 110490.000 95974.8081  13.1%  21.7 1382s
 585676 299603 105647.959   38   95 110490.000 95978.0639  13.1%  21.7 1387s
 587069 300301 101098.686   30   95 110490.000 95981.4313  13.1%  21.7 1391s

...

 11791  3636 1131.64948   20  157 1224.00000 1102.83505  9.90%   6.8  100s
 12548  4033 1187.50172   57  193 1224.00000 1102.83505  9.90%   7.2  105s
 13861  4379 1195.37769  180  166 1224.00000 1102.83505  9.90%   7.6  110s
 15230  5529 1210.31100   50  182 1224.00000 1105.12371  9.71%   8.1  116s
 16383  6303 1144.29553   25  137 1224.00000 1112.46392  9.11%   8.5  120s
 17421  7082 1198.25430   44   78 1224.00000 1112.46392  9.11%   8.9  125s
 19022  7901 1209.59622  183  143 1224.00000 1112.46392  9.11%   9.1  130s
 19298  7910 1210.70962  206  102 1224.00000 1112.46392  9.11%   9.1  138s
H19302  6599                    1212.0000000 1112.46392  8.21%   9.1  138s
 19311  6601     cutoff  206      1212.00000 1112.46392  8.21%   9.1  161s
H19316  6443                    1211.0000000 1112.46392  8.14%   9.1  161s
 19773  6946 1202.02749   39  191 1211.00000 1120.60825  7.46%   9.3  166s
 20661  7558 infeasible   35      1211.00000 1121.78351  7.37%   9.8  172s
 21256  7808 1202.05326  

 139886 48878     cutoff   36      1211.00000 1161.83505  4.06%  20.8  721s
 141197 49315 1205.09794   41  131 1211.00000 1161.94845  4.05%  20.8  726s
 142541 49350 1176.37715   28  197 1211.00000 1162.01632  4.04%  20.8  757s
 142589 49831 1202.28436   32  198 1211.00000 1162.01718  4.04%  20.8  763s
 143848 50234 1183.84107   29  199 1211.00000 1162.05326  4.04%  20.8  769s
 145226 50623 1204.78694   28  157 1211.00000 1162.13058  4.04%  20.8  775s
 146647 51011     cutoff   34      1211.00000 1162.20275  4.03%  20.8  783s
 148102 51274 infeasible   37      1211.00000 1162.30584  4.02%  20.9  789s
 149481 51600 1199.31271   33  145 1211.00000 1162.40722  4.01%  20.9  794s
 150781 51832 1168.55155   34  147 1211.00000 1162.56357  4.00%  20.9  800s
 151991 52311 1167.63230   31  144 1211.00000 1162.72852  3.99%  20.9  805s
 153346 52697 1186.88660   33   90 1211.00000 1162.83505  3.98%  20.9  811s
 154766 53162     cutoff   35      1211.00000 1162.89691  3.97%  20.9  817s
 156108 5353

 287934 85719 1196.96134   32  144 1211.00000 1169.18213  3.45%  21.4 1376s
 289450 86086 1195.94158   34  241 1211.00000 1169.21134  3.45%  21.4 1382s
 290911 86379 1191.86197   35  189 1211.00000 1169.26117  3.45%  21.4 1388s
 292357 86655 1199.77320   33  130 1211.00000 1169.30584  3.44%  21.4 1394s
 293840 86967     cutoff   31      1211.00000 1169.36976  3.44%  21.4 1401s
 295360 87227 infeasible   34      1211.00000 1169.40893  3.43%  21.4 1407s
 296756 87452 1205.53608   34  206 1211.00000 1169.47079  3.43%  21.4 1412s
 298092 87733 1185.52609   32  218 1211.00000 1169.55326  3.42%  21.4 1418s
 299461 88085 1199.84278   37  146 1211.00000 1169.63402  3.42%  21.4 1423s
 300865 88456 1205.89072   32  203 1211.00000 1169.70790  3.41%  21.4 1429s
 302381 88814 1187.71198   31  179 1211.00000 1169.76117  3.41%  21.4 1435s
 303733 89265 1182.08763   33  138 1211.00000 1169.80928  3.40%  21.4 1441s
 305282 89591 1179.79381   34  128 1211.00000 1169.83162  3.40%  21.4 1447s
 306839 8994

 449589 119915     cutoff   41      1211.00000 1173.42955  3.10%  21.5 2059s
 450916 120188 1196.37113   36  125 1211.00000 1173.48454  3.10%  21.5 2065s
 452393 120518     cutoff   31      1211.00000 1173.53608  3.09%  21.5 2071s
 453917 120528 1198.11684   36  123 1211.00000 1173.59450  3.09%  21.5 2084s
 453931 120749 1198.15808   37  109 1211.00000 1173.59536  3.09%  21.5 2091s
 455322 121158 infeasible   37      1211.00000 1173.65636  3.08%  21.5 2098s
 456805 121554 1196.17526   33  152 1211.00000 1173.69072  3.08%  21.5 2106s
 458448 121989 1196.82517   33  154 1211.00000 1173.72509  3.08%  21.5 2116s
 460157 122286 1192.58076   30  207 1211.00000 1173.75773  3.08%  21.5 2124s
 461815 122529 1202.62887   38   75 1211.00000 1173.79038  3.07%  21.5 2131s
 463247 122982     cutoff   32      1211.00000 1173.81443  3.07%  21.5 2140s
 465099 123415 infeasible   39      1211.00000 1173.83986  3.07%  21.5 2149s
 466756 123936 1187.73711   30  138 1211.00000 1173.87113  3.07%  21.5 2157s

 623570 151694     cutoff   41      1211.00000 1176.61512  2.84%  21.4 2955s
 625335 151926     cutoff   43      1211.00000 1176.65979  2.84%  21.4 2963s
 627064 152371     cutoff   33      1211.00000 1176.66753  2.84%  21.4 2971s
 628794 152802 1199.61684   33  162 1211.00000 1176.70619  2.83%  21.4 2980s
 630528 153259     cutoff   34      1211.00000 1176.73196  2.83%  21.4 2989s
 631810 153259 1177.01753   33  157 1211.00000 1176.73703  2.83%  21.4 2990s
 632486 153678 infeasible   37      1211.00000 1176.75945  2.83%  21.4 2998s
 634342 154121     cutoff   40      1211.00000 1176.77629  2.83%  21.4 3008s
 636219 154530 1197.41237   39  120 1211.00000 1176.79897  2.82%  21.4 3017s
 638129 154905 infeasible   40      1211.00000 1176.81186  2.82%  21.4 3025s
 640107 155156 1196.57560   31  145 1211.00000 1176.83162  2.82%  21.4 3034s
 641910 155540 1203.78991   36  181 1211.00000 1176.84880  2.82%  21.4 3045s
 643900 155923 1192.27320   41  113 1211.00000 1176.87371  2.82%  21.3 3053s

In [15]:
# FORMULATION 3
params = {'MIPGap':0, 'TimeLimit':3600, 'Cuts':0}
run_f3('3c_f3', params, datasets)

Set parameter Username
Set parameter LicenseID to value 2634667
Academic license - for non-commercial use only - expires 2026-03-10
Set parameter LogFile to value "logs/3c_f3_bays29.log"
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 3600
Set parameter Cuts to value 0
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 24419 rows, 25230 columns and 92711 nonzeros
Model fingerprint: 0x1712e753
Variable types: 24389 continuous, 841 integer (841 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1625 rows and 2466 columns
Presolve time: 0.52s
Presolved: 22794 rows, 22764 columns, 88648 nonzeros
Variable types: 21952 continuous, 8

Set parameter LicenseID to value 2634667
Academic license - for non-commercial use only - expires 2026-03-10
Set parameter LogFile to value "logs/3c_f3_dantzig42.log"
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 3600
Set parameter Cuts to value 0
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 74131 rows, 75852 columns and 286060 nonzeros
Model fingerprint: 0x2b3b1d3e
Variable types: 74088 continuous, 1764 integer (1764 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 3445 rows and 5209 columns
Presolve time: 2.41s
Presolved: 70686 rows, 70643 columns, 277447 nonzeros
Variable types: 68921 continuous, 1722 integer (172

Set parameter Username
Set parameter LicenseID to value 2634667
Academic license - for non-commercial use only - expires 2026-03-10
Set parameter LogFile to value "logs/3c_f3_pr76.log"
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 3600
Set parameter Cuts to value 0
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 439053 rows, 444752 columns and 1721778 nonzeros
Model fingerprint: 0xe3d5fa2a
Variable types: 438976 continuous, 5776 integer (5776 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+02, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 5776 rows and 11552 columns (presolve time = 5s) ...
Presolve removed 5776 rows and 11552 columns (presolve time = 10s) ...
Presolv

Elapsed ordering time = 30s
Elapsed ordering time = 35s
Elapsed ordering time = 40s
Ordering time: 55.95s

Barrier statistics:
 AA' NZ     : 5.120e+07
 Factor NZ  : 2.793e+08 (roughly 3.0 GB of memory)
 Factor Ops : 3.715e+11 (roughly 4 seconds per iteration)
 Threads    : 10

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.47899205e+06 -4.88737686e+06  1.75e+02 0.00e+00  1.09e+01   172s
   1   4.20231243e+05 -2.06400505e+06  4.90e+01 4.41e-13  2.98e+00   179s
   2   7.15696375e+04 -2.52585847e+05  7.54e+00 7.25e-13  3.83e-01   191s
   3   8.98686670e+03 -5.19460731e+04  1.31e-01 6.68e-13  2.55e-02   202s
   4   7.38564093e+03 -4.79545608e+03  5.59e-03 7.96e-13  4.31e-03   215s
   5   4.98185841e+03 -1.28493340e+03  2.72e-04 7.39e-13  2.20e-03   226s
   6   3.90741304e+03 -1.66840386e+02  1.53e-04 2.05e-12  1.43e-03   237s
   7   3.27860898e+03  4.41928794e+02  1.07e-04 3.10e-12  9.98e-04   247s
   8

f[6,5,6]: 1.0
f[7,5,6]: 1.0
f[7,6,7]: 1.0
f[8,5,6]: 1.0
f[8,6,7]: 1.0
f[8,7,8]: 1.0
f[9,5,6]: 1.0
f[9,6,7]: 1.0
f[9,7,8]: 1.0
f[9,8,17]: 1.0
f[9,10,9]: 1.0
f[9,11,10]: 1.0
f[9,12,11]: 1.0
f[9,14,22]: 1.0
f[9,15,14]: 1.0
f[9,16,15]: 1.0
f[9,17,16]: 1.0
f[9,18,19]: 1.0
f[9,19,20]: 1.0
f[9,20,21]: 1.0
f[9,21,12]: 1.0
f[9,22,23]: 1.0
f[9,23,24]: 1.0
f[9,24,25]: 1.0
f[9,25,26]: 1.0
f[9,26,34]: 1.0
f[9,27,18]: 1.0
f[9,28,36]: 1.0
f[9,29,28]: 1.0
f[9,30,29]: 1.0
f[9,31,30]: 1.0
f[9,32,31]: 1.0
f[9,33,32]: 1.0
f[9,34,35]: 1.0
f[9,35,43]: 1.0
f[9,36,27]: 1.0
f[9,37,47]: 1.0
f[9,38,39]: 1.0
f[9,39,40]: 1.0
f[9,40,41]: 1.0
f[9,41,42]: 1.0
f[9,42,33]: 1.0
f[9,43,44]: 1.0
f[9,44,53]: 1.0
f[9,45,46]: 1.0
f[9,46,37]: 1.0
f[9,47,38]: 1.0
f[9,48,59]: 1.0
f[9,49,48]: 1.0
f[9,50,49]: 1.0
f[9,51,50]: 1.0
f[9,52,51]: 1.0
f[9,53,52]: 1.0
f[9,54,45]: 1.0
f[9,55,64]: 1.0
f[9,56,55]: 1.0
f[9,57,56]: 1.0
f[9,58,57]: 1.0
f[9,59,60]: 1.0
f[9,60,61]: 1.0
f[9,61,62]: 1.0
f[9,62,71]: 1.0
f[9,63,54]: 1.0
f[9,64,63]: 

f[27,5,6]: 1.0
f[27,6,7]: 1.0
f[27,7,8]: 1.0
f[27,8,17]: 1.0
f[27,14,22]: 1.0
f[27,15,14]: 1.0
f[27,16,15]: 1.0
f[27,17,16]: 1.0
f[27,22,23]: 1.0
f[27,23,24]: 1.0
f[27,24,25]: 1.0
f[27,25,26]: 1.0
f[27,26,34]: 1.0
f[27,28,36]: 1.0
f[27,29,28]: 1.0
f[27,30,29]: 1.0
f[27,31,30]: 1.0
f[27,32,31]: 1.0
f[27,33,32]: 1.0
f[27,34,35]: 1.0
f[27,35,43]: 1.0
f[27,36,27]: 1.0
f[27,37,47]: 1.0
f[27,38,39]: 1.0
f[27,39,40]: 1.0
f[27,40,41]: 1.0
f[27,41,42]: 1.0
f[27,42,33]: 1.0
f[27,43,44]: 1.0
f[27,44,53]: 1.0
f[27,45,46]: 1.0
f[27,46,37]: 1.0
f[27,47,38]: 1.0
f[27,48,59]: 1.0
f[27,49,48]: 1.0
f[27,50,49]: 1.0
f[27,51,50]: 1.0
f[27,52,51]: 1.0
f[27,53,52]: 1.0
f[27,54,45]: 1.0
f[27,55,64]: 1.0
f[27,56,55]: 1.0
f[27,57,56]: 1.0
f[27,58,57]: 1.0
f[27,59,60]: 1.0
f[27,60,61]: 1.0
f[27,61,62]: 1.0
f[27,62,71]: 1.0
f[27,63,54]: 1.0
f[27,64,63]: 1.0
f[27,65,66]: 1.0
f[27,66,67]: 1.0
f[27,67,58]: 1.0
f[27,68,77]: 1.0
f[27,69,68]: 1.0
f[27,70,69]: 1.0
f[27,71,80]: 1.0
f[27,72,73]: 1.0
f[27,73,74]: 1.0
f[27

f[36,54,45]: 1.0
f[36,55,64]: 1.0
f[36,56,55]: 1.0
f[36,57,56]: 1.0
f[36,58,57]: 1.0
f[36,59,60]: 1.0
f[36,60,61]: 1.0
f[36,61,62]: 1.0
f[36,62,71]: 1.0
f[36,63,54]: 1.0
f[36,64,63]: 1.0
f[36,65,66]: 1.0
f[36,66,67]: 1.0
f[36,67,58]: 1.0
f[36,68,77]: 1.0
f[36,69,68]: 1.0
f[36,70,69]: 1.0
f[36,71,80]: 1.0
f[36,72,73]: 1.0
f[36,73,74]: 1.0
f[36,74,65]: 1.0
f[36,75,83]: 1.0
f[36,76,75]: 1.0
f[36,77,76]: 1.0
f[36,78,88]: 1.0
f[36,79,70]: 1.0
f[36,80,79]: 1.0
f[36,81,72]: 1.0
f[36,82,81]: 1.0
f[36,83,84]: 1.0
f[36,84,85]: 1.0
f[36,85,86]: 1.0
f[36,86,87]: 1.0
f[36,87,78]: 1.0
f[36,88,89]: 1.0
f[36,89,98]: 1.0
f[36,90,82]: 1.0
f[36,91,90]: 1.0
f[36,92,91]: 1.0
f[36,93,92]: 1.0
f[36,94,93]: 1.0
f[36,95,94]: 1.0
f[36,96,95]: 1.0
f[36,97,96]: 1.0
f[36,98,97]: 1.0
f[37,5,6]: 1.0
f[37,6,7]: 1.0
f[37,7,8]: 1.0
f[37,8,17]: 1.0
f[37,14,22]: 1.0
f[37,15,14]: 1.0
f[37,16,15]: 1.0
f[37,17,16]: 1.0
f[37,22,23]: 1.0
f[37,23,24]: 1.0
f[37,24,25]: 1.0
f[37,25,26]: 1.0
f[37,26,34]: 1.0
f[37,34,35]: 1.0
f[37

f[46,43,44]: 1.0
f[46,44,53]: 1.0
f[46,45,46]: 1.0
f[46,48,59]: 1.0
f[46,49,48]: 1.0
f[46,50,49]: 1.0
f[46,51,50]: 1.0
f[46,52,51]: 1.0
f[46,53,52]: 1.0
f[46,54,45]: 1.0
f[46,55,64]: 1.0
f[46,56,55]: 1.0
f[46,57,56]: 1.0
f[46,58,57]: 1.0
f[46,59,60]: 1.0
f[46,60,61]: 1.0
f[46,61,62]: 1.0
f[46,62,71]: 1.0
f[46,63,54]: 1.0
f[46,64,63]: 1.0
f[46,65,66]: 1.0
f[46,66,67]: 1.0
f[46,67,58]: 1.0
f[46,68,77]: 1.0
f[46,69,68]: 1.0
f[46,70,69]: 1.0
f[46,71,80]: 1.0
f[46,72,73]: 1.0
f[46,73,74]: 1.0
f[46,74,65]: 1.0
f[46,75,83]: 1.0
f[46,76,75]: 1.0
f[46,77,76]: 1.0
f[46,78,88]: 1.0
f[46,79,70]: 1.0
f[46,80,79]: 1.0
f[46,81,72]: 1.0
f[46,82,81]: 1.0
f[46,83,84]: 1.0
f[46,84,85]: 1.0
f[46,85,86]: 1.0
f[46,86,87]: 1.0
f[46,87,78]: 1.0
f[46,88,89]: 1.0
f[46,89,98]: 1.0
f[46,90,82]: 1.0
f[46,91,90]: 1.0
f[46,92,91]: 1.0
f[46,93,92]: 1.0
f[46,94,93]: 1.0
f[46,95,94]: 1.0
f[46,96,95]: 1.0
f[46,97,96]: 1.0
f[46,98,97]: 1.0
f[47,5,6]: 1.0
f[47,6,7]: 1.0
f[47,7,8]: 1.0
f[47,8,17]: 1.0
f[47,14,22]: 1.0
f[47

f[67,75,83]: 1.0
f[67,76,75]: 1.0
f[67,77,76]: 1.0
f[67,78,88]: 1.0
f[67,79,70]: 1.0
f[67,80,79]: 1.0
f[67,81,72]: 1.0
f[67,82,81]: 1.0
f[67,83,84]: 1.0
f[67,84,85]: 1.0
f[67,85,86]: 1.0
f[67,86,87]: 1.0
f[67,87,78]: 1.0
f[67,88,89]: 1.0
f[67,89,98]: 1.0
f[67,90,82]: 1.0
f[67,91,90]: 1.0
f[67,92,91]: 1.0
f[67,93,92]: 1.0
f[67,94,93]: 1.0
f[67,95,94]: 1.0
f[67,96,95]: 1.0
f[67,97,96]: 1.0
f[67,98,97]: 1.0
f[68,5,6]: 1.0
f[68,6,7]: 1.0
f[68,7,8]: 1.0
f[68,8,17]: 1.0
f[68,14,22]: 1.0
f[68,15,14]: 1.0
f[68,16,15]: 1.0
f[68,17,16]: 1.0
f[68,22,23]: 1.0
f[68,23,24]: 1.0
f[68,24,25]: 1.0
f[68,25,26]: 1.0
f[68,26,34]: 1.0
f[68,34,35]: 1.0
f[68,35,43]: 1.0
f[68,43,44]: 1.0
f[68,44,53]: 1.0
f[68,48,59]: 1.0
f[68,49,48]: 1.0
f[68,50,49]: 1.0
f[68,51,50]: 1.0
f[68,52,51]: 1.0
f[68,53,52]: 1.0
f[68,59,60]: 1.0
f[68,60,61]: 1.0
f[68,61,62]: 1.0
f[68,62,71]: 1.0
f[68,69,68]: 1.0
f[68,70,69]: 1.0
f[68,71,80]: 1.0
f[68,79,70]: 1.0
f[68,80,79]: 1.0
f[69,5,6]: 1.0
f[69,6,7]: 1.0
f[69,7,8]: 1.0
f[69,8,17]

f[87,51,50]: 1.0
f[87,52,51]: 1.0
f[87,53,52]: 1.0
f[87,59,60]: 1.0
f[87,60,61]: 1.0
f[87,61,62]: 1.0
f[87,62,71]: 1.0
f[87,68,77]: 1.0
f[87,69,68]: 1.0
f[87,70,69]: 1.0
f[87,71,80]: 1.0
f[87,75,83]: 1.0
f[87,76,75]: 1.0
f[87,77,76]: 1.0
f[87,79,70]: 1.0
f[87,80,79]: 1.0
f[87,83,84]: 1.0
f[87,84,85]: 1.0
f[87,85,86]: 1.0
f[87,86,87]: 1.0
f[88,5,6]: 1.0
f[88,6,7]: 1.0
f[88,7,8]: 1.0
f[88,8,17]: 1.0
f[88,14,22]: 1.0
f[88,15,14]: 1.0
f[88,16,15]: 1.0
f[88,17,16]: 1.0
f[88,22,23]: 1.0
f[88,23,24]: 1.0
f[88,24,25]: 1.0
f[88,25,26]: 1.0
f[88,26,34]: 1.0
f[88,34,35]: 1.0
f[88,35,43]: 1.0
f[88,43,44]: 1.0
f[88,44,53]: 1.0
f[88,48,59]: 1.0
f[88,49,48]: 1.0
f[88,50,49]: 1.0
f[88,51,50]: 1.0
f[88,52,51]: 1.0
f[88,53,52]: 1.0
f[88,59,60]: 1.0
f[88,60,61]: 1.0
f[88,61,62]: 1.0
f[88,62,71]: 1.0
f[88,68,77]: 1.0
f[88,69,68]: 1.0
f[88,70,69]: 1.0
f[88,71,80]: 1.0
f[88,75,83]: 1.0
f[88,76,75]: 1.0
f[88,77,76]: 1.0
f[88,78,88]: 1.0
f[88,79,70]: 1.0
f[88,80,79]: 1.0
f[88,83,84]: 1.0
f[88,84,85]: 1.0
f[88

#### d) with both `presolve` and `cuts` turned off

In [16]:
# FORMULATION 2
params = {'MIPGap':0, 'TimeLimit':3600, 'Presolve':0, 'Cuts':0}
run_f2('3d_f2', params, datasets)

Set parameter Username
Set parameter LicenseID to value 2634667
Academic license - for non-commercial use only - expires 2026-03-10
Set parameter LogFile to value "logs/3d_f2_bays29.log"
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 3600
Set parameter Presolve to value 0
Set parameter Cuts to value 0
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 928 rows, 870 columns and 4064 nonzeros
Model fingerprint: 0xe20bd28c
Variable types: 0 continuous, 870 integer (841 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [3e+01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Variable types: 0 continuous, 870 integer (841 binary)

Root relaxation: objective 1.774000e+03, 118 iterations, 0.00 seconds (0

 223058 53963     cutoff   25       699.00000  658.99206  5.72%  13.6  158s
 225054 53952 infeasible   26       699.00000  659.10714  5.71%  13.6  160s
 232757 53259  660.98413   27   51  699.00000  659.76190  5.61%  13.7  166s
 237562 53048  667.28571   32   21  699.00000  660.17460  5.55%  13.8  170s
 246183 52099     cutoff   28       699.00000  660.95238  5.44%  13.9  176s
 252515 51188  668.44444   46   27  699.00000  661.57143  5.35%  14.1  180s
 261114 50015 infeasible   33       699.00000  662.34921  5.24%  14.2  186s
 267284 48797     cutoff   35       699.00000  662.98810  5.15%  14.3  190s
 273722 47399     cutoff   24       699.00000  663.54762  5.07%  14.4  195s
 281718 44908  667.46032   24   57  699.00000  664.57857  4.92%  14.5  201s
 287938 42859     cutoff   48       699.00000  665.26190  4.83%  14.6  205s
 293832 40797     cutoff   27       699.00000  666.25000  4.69%  14.7  210s
 301393 36703 infeasible   33       699.00000  667.44444  4.51%  14.9  216s
 307516 3307

 114304 66768 98863.3377   33  104 108320.000 92928.7456  14.2%  18.1  221s
 117359 68301     cutoff   41      108320.000 92971.1140  14.2%  18.1  225s
 121714 70619 107344.998   45   68 108320.000 93015.4013  14.1%  18.2  230s
 125673 72798 104789.355   38   49 108320.000 93065.9605  14.1%  18.3  235s
 129585 74530 97722.1118   30  100 108320.000 93112.0132  14.0%  18.3  240s
 133367 76849 infeasible   43      108320.000 93137.7039  14.0%  18.4  245s
 137512 78747 101447.325   30   99 108320.000 93184.1382  14.0%  18.4  250s
 141301 80787 104472.276   42  104 108320.000 93212.2237  13.9%  18.5  255s
 145149 82209 107581.447   45   80 108320.000 93247.5351  13.9%  18.5  261s
H145150 82092                    108264.00000 93247.5351  13.9%  18.5  261s
H145186 81915                    108183.00000 93247.5351  13.8%  18.5  261s
H145246 81875                    108158.99994 93247.5351  13.8%  18.5  261s
 145530 84270 107627.355   46   71 108159.000 93249.8947  13.8%  18.5  271s
 150310 8427

 351112 186999 106280.645   47   92 108159.000 94435.6546  12.7%  17.7  748s
 352277 187617     cutoff   45      108159.000 94438.6184  12.7%  17.7  752s
 353602 188269 96960.3750   28  117 108159.000 94439.0461  12.7%  17.7  756s
 354861 188998 97444.3596   31   91 108159.000 94447.0614  12.7%  17.7  760s
 356222 189618     cutoff   45      108159.000 94452.2434  12.7%  17.7  765s
 358910 190941     cutoff   41      108159.000 94466.3816  12.7%  17.7  773s
 360158 191474 infeasible   50      108159.000 94467.9474  12.7%  17.7  777s
 361398 192083 103013.038   40  106 108159.000 94471.0066  12.7%  17.7  782s
 362671 192825 104841.434   38   91 108159.000 94477.7675  12.6%  17.7  786s
 363963 193471 102727.818   37   71 108159.000 94480.9671  12.6%  17.7  791s
 365228 194006 101088.927   32  141 108159.000 94485.6447  12.6%  17.7  795s
 367721 195384 99686.7281   35   80 108159.000 94497.5899  12.6%  17.7  803s
 368998 196102 103337.875   43   99 108159.000 94503.1184  12.6%  17.7  808s

 505542 258333     cutoff   45      108159.000 94969.8257  12.2%  18.6 1402s
 506863 258913 104151.853   41   92 108159.000 94971.6382  12.2%  18.6 1406s
 508098 259597     cutoff   44      108159.000 94976.1776  12.2%  18.6 1412s
 509499 260122     cutoff   44      108159.000 94980.4737  12.2%  18.6 1418s
 510875 260726 97967.7566   35  124 108159.000 94983.8289  12.2%  18.6 1424s
 512303 261295 99996.0921   36   82 108159.000 94988.8947  12.2%  18.6 1430s
 513707 261705 105453.658   38   97 108159.000 94994.2193  12.2%  18.6 1436s
 514687 262367     cutoff   41      108159.000 94998.4474  12.2%  18.6 1441s
 515945 263050 104613.789   39   79 108159.000 95003.1316  12.2%  18.6 1448s
 517427 263683 102485.864   35  144 108159.000 95004.0789  12.2%  18.6 1455s
 518934 264176 104929.092   40   87 108159.000 95010.9013  12.2%  18.6 1462s
 520451 264694 105262.658   43   93 108159.000 95013.4211  12.2%  18.6 1469s
 521822 265281 95987.9474   29   73 108159.000 95017.7632  12.1%  18.7 1475s

...

 797557 378983 101733.416   42  120 108159.000 95677.7406  11.5%  19.6 3121s
 799105 379484 106713.684   40   72 108159.000 95682.1908  11.5%  19.6 3130s
 800520 380140 infeasible   40      108159.000 95685.2928  11.5%  19.6 3140s
 802042 380749 99072.7456   33   85 108159.000 95688.7281  11.5%  19.6 3151s
 803467 381286 106417.807   50   54 108159.000 95690.9868  11.5%  19.6 3161s
 805108 381590 102909.855   37   96 108159.000 95693.2325  11.5%  19.6 3170s
 806046 381986 96845.0789   31  108 108159.000 95694.9825  11.5%  19.6 3182s
 807575 382595 106094.847   40  132 108159.000 95697.7368  11.5%  19.6 3193s
 809056 383097 97035.1996   31   97 108159.000 95700.2237  11.5%  19.6 3202s
 810349 383634 99811.9167   37  112 108159.000 95703.6184  11.5%  19.6 3214s
 811907 384290 101958.368   35  102 108159.000 95706.2588  11.5%  19.6 3224s
 813351 384842 100988.145   33   71 108159.000 95709.1184  11.5%  19.6 3235s
 814881 385387 104165.548   37  114 108159.000 95711.9770  11.5%  19.7 3246s

 36381 17816     cutoff   35      1211.00000 1150.08081  5.03%  26.4  203s
 37802 18333 1185.75758   32  137 1211.00000 1150.22727  5.02%  26.5  208s
 39113 18858 1173.98990   26  136 1211.00000 1150.47111  5.00%  26.6  214s
 40311 19296 1195.51717   29  115 1211.00000 1151.13131  4.94%  26.7  219s
 41424 19821 1202.81818   35  108 1211.00000 1151.37374  4.92%  26.7  225s
 42780 20277 1200.16932   38  163 1211.00000 1151.93835  4.88%  26.8  230s
 44046 20799 infeasible   32      1211.00000 1152.10101  4.86%  26.8  236s
 45370 21181 infeasible   35      1211.00000 1152.22222  4.85%  26.9  241s
 46735 21659 1204.78047   31  137 1211.00000 1152.43434  4.84%  26.9  247s
 48056 22132 1196.54545   31  145 1211.00000 1153.03535  4.79%  26.9  252s
 49429 22472 infeasible   33      1211.00000 1153.32828  4.76%  26.9  257s
 50535 23147 1199.68723   31  130 1211.00000 1153.83670  4.72%  26.9  262s
 51789 23540 infeasible   45      1211.00000 1153.99394  4.71%  27.0  267s
 53144 23962 infeasible  

 190294 65215 1204.07037   32  146 1211.00000 1168.12121  3.54%  27.6  961s
 191721 65436 1196.73350   36  164 1211.00000 1168.18687  3.54%  27.6  967s
 193120 65750 1193.09630   28  135 1211.00000 1168.26768  3.53%  27.6  973s
 194404 66003 1188.03030   39   79 1211.00000 1168.38333  3.52%  27.6  978s
 195705 66358     cutoff   33      1211.00000 1168.55556  3.50%  27.6  984s
 197067 66807 1192.65993   32   96 1211.00000 1168.63805  3.50%  27.6  990s
 198320 67269 1184.89192   30  142 1211.00000 1168.69798  3.49%  27.6  997s
 199750 67736     cutoff   30      1211.00000 1168.75556  3.49%  27.6 1003s
 201216 68184 1184.65741   29  120 1211.00000 1168.79798  3.48%  27.6 1010s
 202663 68688 1176.86869   27  116 1211.00000 1168.84271  3.48%  27.6 1017s
 204282 69073 1176.02778   27  169 1211.00000 1168.88889  3.48%  27.6 1025s
 205718 69479 1194.82492   35  115 1211.00000 1168.93468  3.47%  27.6 1031s
 207164 69861 1206.78451   29  131 1211.00000 1168.97980  3.47%  27.6 1038s
 208623 7033

 343704 102382     cutoff   32      1211.00000 1173.84175  3.07%  27.4 1899s
 345406 102741     cutoff   32      1211.00000 1173.86532  3.07%  27.4 1910s
 347014 103007 1183.72727   28  115 1211.00000 1173.88805  3.06%  27.4 1921s
 348694 103362     cutoff   36      1211.00000 1173.90909  3.06%  27.4 1931s
 350286 103766 1183.04040   36  152 1211.00000 1173.94276  3.06%  27.4 1943s
 351981 104128 1187.84848   28  116 1211.00000 1173.96970  3.06%  27.4 1953s
 353500 104374     cutoff   39      1211.00000 1174.00067  3.06%  27.4 1964s
 354927 104678 1187.94444   30  112 1211.00000 1174.04040  3.05%  27.4 1974s
 356439 104964     cutoff   36      1211.00000 1174.08081  3.05%  27.4 1984s
 357963 105185 infeasible   41      1211.00000 1174.13131  3.04%  27.4 1993s
 359194 105394     cutoff   37      1211.00000 1174.16162  3.04%  27.4 2002s
 360666 105620 1194.56890   36  187 1211.00000 1174.21549  3.04%  27.4 2012s
 362169 105880 1190.49158   39  131 1211.00000 1174.28283  3.03%  27.4 2023s

 513289 136537 1199.98653   35  127 1211.00000 1177.54882  2.76%  27.2 3260s
 514937 136951 1197.80892   39  141 1211.00000 1177.57828  2.76%  27.2 3275s
 516708 137322 1200.47691   41  125 1211.00000 1177.61145  2.76%  27.2 3290s
 518406 137733 1200.68687   35   71 1211.00000 1177.63636  2.76%  27.2 3307s
 520221 138165 infeasible   40      1211.00000 1177.66330  2.75%  27.2 3323s
 522099 138537 1192.64899   31  104 1211.00000 1177.68620  2.75%  27.2 3342s
 523938 138988 1203.52413   29  139 1211.00000 1177.70623  2.75%  27.2 3359s
 525854 139455 infeasible   36      1211.00000 1177.72727  2.75%  27.2 3376s
 527736 139870 1191.77908   28  150 1211.00000 1177.75253  2.75%  27.2 3393s
 529641 140294 1188.56465   32  150 1211.00000 1177.77104  2.74%  27.2 3410s
 531466 140642     cutoff   34      1211.00000 1177.78884  2.74%  27.2 3428s
 533395 140990 1204.78451   34  118 1211.00000 1177.80976  2.74%  27.2 3448s
 535377 141283 1201.77104   33   83 1211.00000 1177.83165  2.74%  27.2 3466s

In [17]:
# FORMULATION 3
params = {'MIPGap':0, 'TimeLimit':3600, 'Presolve':0, 'Cuts':0}
run_f3('3d_f3', params, datasets)

Set parameter Username
Set parameter LicenseID to value 2634667
Academic license - for non-commercial use only - expires 2026-03-10
Set parameter LogFile to value "logs/3d_f3_bays29.log"
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 3600
Set parameter Presolve to value 0
Set parameter Cuts to value 0
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 24419 rows, 25230 columns and 92711 nonzeros
Model fingerprint: 0x1712e753
Variable types: 24389 continuous, 841 integer (841 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Variable types: 24389 continuous, 841 integer (841 binary)

Root relaxation: objective 2.013500e+03, 16234 iterations, 

Set parameter LicenseID to value 2634667
Academic license - for non-commercial use only - expires 2026-03-10
Set parameter LogFile to value "logs/3d_f3_dantzig42.log"
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 3600
Set parameter Presolve to value 0
Set parameter Cuts to value 0
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 74131 rows, 75852 columns and 286060 nonzeros
Model fingerprint: 0x2b3b1d3e
Variable types: 74088 continuous, 1764 integer (1764 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Variable types: 74088 continuous, 1764 integer (1764 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing pri

Set parameter LicenseID to value 2634667
Academic license - for non-commercial use only - expires 2026-03-10
Set parameter LogFile to value "logs/3d_f3_pr76.log"
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 3600
Set parameter Presolve to value 0
Set parameter Cuts to value 0
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 439053 rows, 444752 columns and 1721778 nonzeros
Model fingerprint: 0xe3d5fa2a
Variable types: 438976 continuous, 5776 integer (5776 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+02, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Variable types: 438976 continuous, 5776 integer (5776 binary)

Deterministic concurrent LP optimizer: primal simplex, dual simplex, and b



Root crossover log...

  943611 DPushes remaining with DInf 0.0000000e+00               302s
    8085 DPushes remaining with DInf 0.0000000e+00               310s
    7909 DPushes remaining with DInf 0.0000000e+00               310s
       0 DPushes remaining with DInf 0.0000000e+00               315s

    5870 PPushes remaining with PInf 0.0000000e+00               315s
       0 PPushes remaining with PInf 0.0000000e+00               317s

  Push phase complete: Pinf 0.0000000e+00, Dinf 8.1055163e-12    317s


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
  422096    1.2060000e+03   0.000000e+00   0.000000e+00    317s
Concurrent spin time: 0.01s

Solved with barrier

Root relaxation: objective 1.206000e+03, 422096 iterations, 311.17 seconds (94.43 work units)
Total elapsed time = 382.92s (DegenMoves)
Total elapsed time = 387.66s (DegenMoves)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth Int

f[16,83,75]: 1.0
f[16,84,83]: 1.0
f[16,85,84]: 1.0
f[16,86,85]: 1.0
f[16,87,86]: 1.0
f[16,88,78]: 1.0
f[16,89,88]: 1.0
f[16,90,91]: 1.0
f[16,91,92]: 1.0
f[16,92,93]: 1.0
f[16,93,94]: 1.0
f[16,94,95]: 1.0
f[16,95,96]: 1.0
f[16,96,97]: 1.0
f[16,97,98]: 1.0
f[16,98,89]: 1.0
f[17,0,9]: 1.0
f[17,1,0]: 1.0
f[17,2,1]: 1.0
f[17,3,2]: 1.0
f[17,4,13]: 1.0
f[17,5,4]: 1.0
f[17,9,10]: 1.0
f[17,10,11]: 1.0
f[17,11,12]: 1.0
f[17,12,21]: 1.0
f[17,13,3]: 1.0
f[17,14,15]: 1.0
f[17,15,16]: 1.0
f[17,16,17]: 1.0
f[17,18,27]: 1.0
f[17,19,18]: 1.0
f[17,20,19]: 1.0
f[17,21,20]: 1.0
f[17,22,14]: 1.0
f[17,23,22]: 1.0
f[17,24,23]: 1.0
f[17,25,24]: 1.0
f[17,26,25]: 1.0
f[17,27,36]: 1.0
f[17,28,29]: 1.0
f[17,29,30]: 1.0
f[17,30,31]: 1.0
f[17,31,32]: 1.0
f[17,32,33]: 1.0
f[17,33,42]: 1.0
f[17,34,26]: 1.0
f[17,35,34]: 1.0
f[17,36,28]: 1.0
f[17,37,46]: 1.0
f[17,38,47]: 1.0
f[17,39,38]: 1.0
f[17,40,39]: 1.0
f[17,41,40]: 1.0
f[17,42,41]: 1.0
f[17,43,35]: 1.0
f[17,44,43]: 1.0
f[17,45,54]: 1.0
f[17,46,45]: 1.0
f[17,47,37

...

f[40,31,32]: 1.0
f[40,32,33]: 1.0
f[40,33,42]: 1.0
f[40,36,28]: 1.0
f[40,41,40]: 1.0
f[40,42,41]: 1.0
f[41,0,9]: 1.0
f[41,1,0]: 1.0
f[41,2,1]: 1.0
f[41,3,2]: 1.0
f[41,4,13]: 1.0
f[41,5,4]: 1.0
f[41,9,10]: 1.0
f[41,10,11]: 1.0
f[41,11,12]: 1.0
f[41,12,21]: 1.0
f[41,13,3]: 1.0
f[41,18,27]: 1.0
f[41,19,18]: 1.0
f[41,20,19]: 1.0
f[41,21,20]: 1.0
f[41,27,36]: 1.0
f[41,28,29]: 1.0
f[41,29,30]: 1.0
f[41,30,31]: 1.0
f[41,31,32]: 1.0
f[41,32,33]: 1.0
f[41,33,42]: 1.0
f[41,36,28]: 1.0
f[41,42,41]: 1.0
f[42,0,9]: 1.0
f[42,1,0]: 1.0
f[42,2,1]: 1.0
f[42,3,2]: 1.0
f[42,4,13]: 1.0
f[42,5,4]: 1.0
f[42,9,10]: 1.0
f[42,10,11]: 1.0
f[42,11,12]: 1.0
f[42,12,21]: 1.0
f[42,13,3]: 1.0
f[42,18,27]: 1.0
f[42,19,18]: 1.0
f[42,20,19]: 1.0
f[42,21,20]: 1.0
f[42,27,36]: 1.0
f[42,28,29]: 1.0
f[42,29,30]: 1.0
f[42,30,31]: 1.0
f[42,31,32]: 1.0
f[42,32,33]: 1.0
f[42,33,42]: 1.0
f[42,36,28]: 1.0
f[43,0,9]: 1.0
f[43,1,0]: 1.0
f[43,2,1]: 1.0
f[43,3,2]: 1.0
f[43,4,13]: 1.0
f[43,5,4]: 1.0
f[43,9,10]: 1.0
f[43,10,11]: 1.0
f

f[50,68,69]: 1.0
f[50,69,70]: 1.0
f[50,70,79]: 1.0
f[50,71,62]: 1.0
f[50,72,81]: 1.0
f[50,73,72]: 1.0
f[50,74,73]: 1.0
f[50,75,76]: 1.0
f[50,76,77]: 1.0
f[50,77,68]: 1.0
f[50,78,87]: 1.0
f[50,79,80]: 1.0
f[50,80,71]: 1.0
f[50,81,82]: 1.0
f[50,82,90]: 1.0
f[50,83,75]: 1.0
f[50,84,83]: 1.0
f[50,85,84]: 1.0
f[50,86,85]: 1.0
f[50,87,86]: 1.0
f[50,88,78]: 1.0
f[50,89,88]: 1.0
f[50,90,91]: 1.0
f[50,91,92]: 1.0
f[50,92,93]: 1.0
f[50,93,94]: 1.0
f[50,94,95]: 1.0
f[50,95,96]: 1.0
f[50,96,97]: 1.0
f[50,97,98]: 1.0
f[50,98,89]: 1.0
f[51,0,9]: 1.0
f[51,1,0]: 1.0
f[51,2,1]: 1.0
f[51,3,2]: 1.0
f[51,4,13]: 1.0
f[51,5,4]: 1.0
f[51,9,10]: 1.0
f[51,10,11]: 1.0
f[51,11,12]: 1.0
f[51,12,21]: 1.0
f[51,13,3]: 1.0
f[51,18,27]: 1.0
f[51,19,18]: 1.0
f[51,20,19]: 1.0
f[51,21,20]: 1.0
f[51,27,36]: 1.0
f[51,28,29]: 1.0
f[51,29,30]: 1.0
f[51,30,31]: 1.0
f[51,31,32]: 1.0
f[51,32,33]: 1.0
f[51,33,42]: 1.0
f[51,36,28]: 1.0
f[51,37,46]: 1.0
f[51,38,47]: 1.0
f[51,39,38]: 1.0
f[51,40,39]: 1.0
f[51,41,40]: 1.0
f[51,42,41

f[66,47,37]: 1.0
f[66,54,63]: 1.0
f[66,55,56]: 1.0
f[66,56,57]: 1.0
f[66,57,58]: 1.0
f[66,58,67]: 1.0
f[66,63,64]: 1.0
f[66,64,55]: 1.0
f[66,67,66]: 1.0
f[67,0,9]: 1.0
f[67,1,0]: 1.0
f[67,2,1]: 1.0
f[67,3,2]: 1.0
f[67,4,13]: 1.0
f[67,5,4]: 1.0
f[67,9,10]: 1.0
f[67,10,11]: 1.0
f[67,11,12]: 1.0
f[67,12,21]: 1.0
f[67,13,3]: 1.0
f[67,18,27]: 1.0
f[67,19,18]: 1.0
f[67,20,19]: 1.0
f[67,21,20]: 1.0
f[67,27,36]: 1.0
f[67,28,29]: 1.0
f[67,29,30]: 1.0
f[67,30,31]: 1.0
f[67,31,32]: 1.0
f[67,32,33]: 1.0
f[67,33,42]: 1.0
f[67,36,28]: 1.0
f[67,37,46]: 1.0
f[67,38,47]: 1.0
f[67,39,38]: 1.0
f[67,40,39]: 1.0
f[67,41,40]: 1.0
f[67,42,41]: 1.0
f[67,45,54]: 1.0
f[67,46,45]: 1.0
f[67,47,37]: 1.0
f[67,54,63]: 1.0
f[67,55,56]: 1.0
f[67,56,57]: 1.0
f[67,57,58]: 1.0
f[67,58,67]: 1.0
f[67,63,64]: 1.0
f[67,64,55]: 1.0
f[68,0,9]: 1.0
f[68,1,0]: 1.0
f[68,2,1]: 1.0
f[68,3,2]: 1.0
f[68,4,13]: 1.0
f[68,5,4]: 1.0
f[68,9,10]: 1.0
f[68,10,11]: 1.0
f[68,11,12]: 1.0
f[68,12,21]: 1.0
f[68,13,3]: 1.0
f[68,18,27]: 1.0
f[68,1

f[77,75,76]: 1.0
f[77,76,77]: 1.0
f[77,78,87]: 1.0
f[77,81,82]: 1.0
f[77,82,90]: 1.0
f[77,83,75]: 1.0
f[77,84,83]: 1.0
f[77,85,84]: 1.0
f[77,86,85]: 1.0
f[77,87,86]: 1.0
f[77,88,78]: 1.0
f[77,89,88]: 1.0
f[77,90,91]: 1.0
f[77,91,92]: 1.0
f[77,92,93]: 1.0
f[77,93,94]: 1.0
f[77,94,95]: 1.0
f[77,95,96]: 1.0
f[77,96,97]: 1.0
f[77,97,98]: 1.0
f[77,98,89]: 1.0
f[78,0,9]: 1.0
f[78,1,0]: 1.0
f[78,2,1]: 1.0
f[78,3,2]: 1.0
f[78,4,13]: 1.0
f[78,5,4]: 1.0
f[78,9,10]: 1.0
f[78,10,11]: 1.0
f[78,11,12]: 1.0
f[78,12,21]: 1.0
f[78,13,3]: 1.0
f[78,18,27]: 1.0
f[78,19,18]: 1.0
f[78,20,19]: 1.0
f[78,21,20]: 1.0
f[78,27,36]: 1.0
f[78,28,29]: 1.0
f[78,29,30]: 1.0
f[78,30,31]: 1.0
f[78,31,32]: 1.0
f[78,32,33]: 1.0
f[78,33,42]: 1.0
f[78,36,28]: 1.0
f[78,37,46]: 1.0
f[78,38,47]: 1.0
f[78,39,38]: 1.0
f[78,40,39]: 1.0
f[78,41,40]: 1.0
f[78,42,41]: 1.0
f[78,45,54]: 1.0
f[78,46,45]: 1.0
f[78,47,37]: 1.0
f[78,54,63]: 1.0
f[78,55,56]: 1.0
f[78,56,57]: 1.0
f[78,57,58]: 1.0
f[78,58,67]: 1.0
f[78,63,64]: 1.0
f[78,64,55

f[91,21,20]: 1.0
f[91,27,36]: 1.0
f[91,28,29]: 1.0
f[91,29,30]: 1.0
f[91,30,31]: 1.0
f[91,31,32]: 1.0
f[91,32,33]: 1.0
f[91,33,42]: 1.0
f[91,36,28]: 1.0
f[91,37,46]: 1.0
f[91,38,47]: 1.0
f[91,39,38]: 1.0
f[91,40,39]: 1.0
f[91,41,40]: 1.0
f[91,42,41]: 1.0
f[91,45,54]: 1.0
f[91,46,45]: 1.0
f[91,47,37]: 1.0
f[91,54,63]: 1.0
f[91,55,56]: 1.0
f[91,56,57]: 1.0
f[91,57,58]: 1.0
f[91,58,67]: 1.0
f[91,63,64]: 1.0
f[91,64,55]: 1.0
f[91,65,74]: 1.0
f[91,66,65]: 1.0
f[91,67,66]: 1.0
f[91,72,81]: 1.0
f[91,73,72]: 1.0
f[91,74,73]: 1.0
f[91,81,82]: 1.0
f[91,82,90]: 1.0
f[91,90,91]: 1.0
f[92,0,9]: 1.0
f[92,1,0]: 1.0
f[92,2,1]: 1.0
f[92,3,2]: 1.0
f[92,4,13]: 1.0
f[92,5,4]: 1.0
f[92,9,10]: 1.0
f[92,10,11]: 1.0
f[92,11,12]: 1.0
f[92,12,21]: 1.0
f[92,13,3]: 1.0
f[92,18,27]: 1.0
f[92,19,18]: 1.0
f[92,20,19]: 1.0
f[92,21,20]: 1.0
f[92,27,36]: 1.0
f[92,28,29]: 1.0
f[92,29,30]: 1.0
f[92,30,31]: 1.0
f[92,31,32]: 1.0
f[92,32,33]: 1.0
f[92,33,42]: 1.0
f[92,36,28]: 1.0
f[92,37,46]: 1.0
f[92,38,47]: 1.0
f[92,39,38